In [1]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)
import logging
import argparse # Argument parsing

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle
import torch
import torch.optim as optim
import copy                    # clone tensor
import time

# Custom imports
import GEMS_TCO
from GEMS_TCO import kernels


import torch.nn.functional as F
from GEMS_TCO import kernels 
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import load_data
from GEMS_TCO import alg_optimization, alg_opt_Encoder
from GEMS_TCO import configuration as config

from typing import Optional, List, Tuple
from pathlib import Path
import typer
import json
from json import JSONEncoder

from GEMS_TCO import configuration as config
from GEMS_TCO import data_preprocess as dmbh
from GEMS_TCO.data_loader import load_data2
import os
from sklearn.neighbors import BallTree
from GEMS_TCO.data_loader import load_data2

Load monthly data

In [2]:
space: List[str] = ['1', '1']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 20
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)


df_map, ord_mm, nns_map = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,
lat_range=[0.0, 5.0],      
lon_range=[123.0, 133.0] 
)

#days: List[str] = ['0', '31']
#days_s_e = [int(d) for d in days]
#days_list = list(range(days_s_e[0], days_s_e[1]))

Subsetting data to lat: [0.0, 5.0], lon: [123.0, 133.0]


In [3]:
daily_aggregated_tensors = [] 
daily_hourly_maps = []        

for day_index in range(31):
  
    hour_start_index = day_index * 8
    #hour_end_index = (day_index + 1) * 8
    hour_end_index = day_index*8 + 1
    hour_indices = [hour_start_index, hour_end_index]
    
    # Load the data for the current day
    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
        df_map, 
        hour_indices, 
        ord_mm= None,  
        dtype=torch.float 
    )
    # Append the day's data to their respective lists
    daily_aggregated_tensors.append(day_aggregated_tensor)
    daily_hourly_maps.append(day_hourly_map) 

print(daily_aggregated_tensors[0].shape)
#print(daily_hourly_maps[0])

torch.Size([18126, 4])


In [4]:

# Assume GEMS_TCO is a custom class/module you have available
# from your_project import GEMS_TCO

# =========================================================================
# 1. Helper Functions
# =========================================================================

def subset_tensor(df_tensor: torch.Tensor) -> torch.Tensor:
    """Subsets a tensor to a specific lat/lon range."""
    #lat_mask = (df_tensor[:, 0] >= -5) & (df_tensor[:, 0] <= 6.3)
    #lon_mask = (df_tensor[:, 1] >= 118) & (df_tensor[:, 1] <= 134.2)
    lat_mask = (df_tensor[:, 0] >= 0) & (df_tensor[:, 0] <= 5)
    lon_mask = (df_tensor[:, 1] >= 123) & (df_tensor[:, 1] <= 133)

    df_sub = df_tensor[lat_mask & lon_mask].clone()
    return df_sub

def apply_first_difference_2d_tensor(df_tensor: torch.Tensor) -> torch.Tensor:
    """
    Applies a 2D first-order difference filter using convolution.
    This approximates Z(s) = [X(s+d_lat) - X(s)] + [X(s+d_lon) - X(s)].
    """
    if df_tensor.size(0) == 0:
        return torch.empty(0, 4)

    # 1. Get grid dimensions and validate
    unique_lats = torch.unique(df_tensor[:, 0])
    unique_lons = torch.unique(df_tensor[:, 1])
    lat_count, lon_count = unique_lats.size(0), unique_lons.size(0)

    if df_tensor.size(0) != lat_count * lon_count:
        raise ValueError("Tensor size does not match grid dimensions. Must be a complete grid.")
    if lat_count < 2 or lon_count < 2:
        return torch.empty(0, 4)

    # 2. Reshape data and define the correct kernel
    ozone_data = df_tensor[:, 2].reshape(1, 1, lat_count, lon_count)
    
    # ✅ CORRECT KERNEL: This kernel results in the standard first-order difference:
    # Z(i,j) = X(i+1,j) + X(i,j+1) - 2*X(i,j)
    # Note: F.conv2d in PyTorch actually performs cross-correlation. To get a true
    # convolution result, the kernel would need to be flipped. However, for a 
    # forward difference operator, defining the kernel for cross-correlation is more direct.
    # The kernel below is designed for cross-correlation to achieve the desired differencing.
    diff_kernel = torch.tensor([[[[-2., 1.],
                                  [ 1., 0.]]]], dtype=torch.float32)

    # 3. Apply convolution (which acts as cross-correlation)
    filtered_grid = F.conv2d(ozone_data, diff_kernel, padding='valid').squeeze()

    # 4. Determine coordinates for the new, smaller grid
    # The new grid corresponds to the anchor points of the kernel
    new_lats = unique_lats[:-1]
    new_lons = unique_lons[:-1]

    # 5. Reconstruct the output tensor
    new_lat_grid, new_lon_grid = torch.meshgrid(new_lats, new_lons, indexing='ij')
    filtered_values = filtered_grid.flatten()
    time_value = df_tensor[0, 3].repeat(filtered_values.size(0))

    new_tensor = torch.stack([
        new_lat_grid.flatten(),
        new_lon_grid.flatten(),
        filtered_values,
        time_value
    ], dim=1)
    
    return new_tensor

# =========================================================================
# 2. Data Loading (Unchanged)
# =========================================================================
# ⚠️ NOTE: You must define these variables
# mac_data_path = "..."
# year = 2022
# month_str = "01"
# class GEMS_TCO: # Placeholder
#     def load_data(self, path): return self
#     def load_working_data_byday_wo_mm(self, data, indices):
#         return {'key': torch.randn(100, 4)}, torch.randn(100, 4)


# =========================================================================
# 3. Main Processing Loop (Unchanged)
# =========================================================================
spatially_filtered_days = []
for day_idx, day_map in enumerate(daily_hourly_maps):
    tensors_to_aggregate = []
    for key, tensor in day_map.items():
        subsetted = subset_tensor(tensor)
        if subsetted.size(0) > 0:
            try:
                diff_applied = apply_first_difference_2d_tensor(subsetted)
                if diff_applied.size(0) > 0:
                    tensors_to_aggregate.append(diff_applied)
            except ValueError as e:
                print(f"Skipping data chunk on day {day_idx+1} due to error: {e}")

    if tensors_to_aggregate:
        aggregated_day_tensor = torch.cat(tensors_to_aggregate, dim=0)
        spatially_filtered_days.append(aggregated_day_tensor)

# =========================================================================
# 4. Verification (Unchanged)
# =========================================================================
print("\n--- Results ---")
print(f"Number of final spatially-differenced day tensors: {len(spatially_filtered_days)}")
if spatially_filtered_days:
    # Save the processed data for the next script
    processed_output_path = "spatial_first_difference_data.pkl"
    with open(processed_output_path, 'wb') as f:
        pickle.dump(spatially_filtered_days, f)
    print(f"Processed data saved to {processed_output_path}")

    print(f"\nShape of the first final tensor: {spatially_filtered_days[0].shape}")
    print("First final tensor head:")
    print(spatially_filtered_days[0][:5])
else:
    print("\nNo final differenced tensors were created.")


--- Results ---
Number of final spatially-differenced day tensors: 31
Processed data saved to spatial_first_difference_data.pkl

Shape of the first final tensor: torch.Size([17854, 4])
First final tensor head:
tensor([[ 4.0000e-03,  1.2303e+02,  2.9422e+00,  2.1000e+01],
        [ 4.0000e-03,  1.2309e+02,  1.9636e+00,  2.1000e+01],
        [ 4.0000e-03,  1.2316e+02, -1.3187e+00,  2.1000e+01],
        [ 4.0000e-03,  1.2322e+02, -3.1683e+00,  2.1000e+01],
        [ 4.0000e-03,  1.2328e+02, -5.4922e-01,  2.1000e+01]])


In [33]:
import torch
import numpy as np
import matplotlib.pyplot as plt 
import cmath
import pickle
import time 
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.nn import Parameter
import torch.fft 
from typing import List, Dict, Any, Callable

# =========================================================================
# 1. Tapering, Autocorrelation, and Covariance Functions
# =========================================================================

def cgn_hamming(u, n1, n2):
    """Computes a 2D Hamming window."""
    u1, u2 = u
    device = u1.device if isinstance(u1, torch.Tensor) else (u2.device if isinstance(u2, torch.Tensor) else torch.device('cpu'))
    u1_tensor = u1.to(device) if isinstance(u1, torch.Tensor) else torch.tensor(u1, device=device, dtype=torch.float32)
    u2_tensor = u2.to(device) if isinstance(u2, torch.Tensor) else torch.tensor(u2, device=device, dtype=torch.float32)

    n1_eff = float(n1) if n1 > 0 else 1.0
    n2_eff = float(n2) if n2 > 0 else 1.0
    hamming1 = 0.54 + 0.46 * torch.cos(2.0 * torch.pi * u1_tensor / n1_eff)
    hamming2 = 0.54 + 0.46 * torch.cos(2.0 * torch.pi * u2_tensor / n2_eff)
    return hamming1 * hamming2

def cgn_2dbartlett(u, n1, n2): # Kept for potential future use
    """Computes a 2D Bartlett window function."""
    u1, u2 = u
    device = u1.device if isinstance(u1, torch.Tensor) else (u2.device if isinstance(u2, torch.Tensor) else torch.device('cpu'))
    u1_tensor = u1.to(device) if isinstance(u1, torch.Tensor) else torch.tensor(u1, device=device, dtype=torch.float32)
    u2_tensor = u2.to(device) if isinstance(u2, torch.Tensor) else torch.tensor(u2, device=device, dtype=torch.float32)
    n1_eff = float(n1) if n1 > 0 else 1.0
    n2_eff = float(n2) if n2 > 0 else 1.0
    kernel = (1.0 - torch.abs(u1_tensor) / n1_eff) * (1.0 - torch.abs(u2_tensor) / n2_eff)
    return torch.clamp(kernel, min=0.0)

def calculate_taper_autocorrelation_fft(taper_grid, n1, n2, device):
    """
    Computes the normalized taper autocorrelation function c_gn(u) using FFT.
    """
    taper_grid = taper_grid.to(device) 
    H = torch.sum(taper_grid**2)
    if H < 1e-12:
        print("Warning: Sum of squared taper weights (H) is near zero.")
        return torch.zeros((2*n1-1, 2*n2-1), device=device, dtype=taper_grid.dtype)

    N1, N2 = 2 * n1 - 1, 2 * n2 - 1
    taper_fft = torch.fft.fft2(taper_grid, s=(N1, N2))
    power_spectrum = torch.abs(taper_fft)**2
    autocorr_unnormalized = torch.fft.ifft2(power_spectrum).real
    autocorr_shifted = torch.fft.fftshift(autocorr_unnormalized)

    c_gn_grid = autocorr_shifted / (H + 1e-12)

    return c_gn_grid 

def cov_x_spatial_model_kernel(u1, u2, t, params):
    """
    Computes autocovariance of X using the SPATIAL-ONLY kernel.
    u1, u2 are PHYSICAL lags (already scaled by deltas).
    
    params index map (4 params):
    [0] log_phi1 (log(sigmasq / range_lon))
    [1] log_phi2 (log(1 / range_lon))
    [2] log_phi3 (log((range_lon / range_lat)^2))
    [3] log_nugget
    """
    device = params.device
    u1_dev = u1.to(device) if isinstance(u1, torch.Tensor) else torch.tensor(u1, device=device, dtype=torch.float32)
    u2_dev = u2.to(device) if isinstance(u2, torch.Tensor) else torch.tensor(u2, device=device, dtype=torch.float32)
    t_dev = t.to(device) if isinstance(t, torch.Tensor) else torch.tensor(t, device=device, dtype=torch.float32)

    if torch.isnan(params).any() or torch.isinf(params).any():
         print("Warning: NaN/Inf in log-params before exp in cov_x_spatial_model_kernel")
         out_shape = torch.broadcast_shapes(u1_dev.shape, u2_dev.shape, t_dev.shape)
         return torch.full(out_shape, float('nan'), device=device, dtype=torch.float32)

    phi1   = torch.exp(params[0])
    phi2   = torch.exp(params[1]) 
    phi3   = torch.exp(params[2]) 
    nugget = torch.exp(params[3])

    epsilon = 1e-12
    sigmasq = phi1 / (phi2 + epsilon)  
    range_lon_inv = phi2
    range_lat_inv = torch.sqrt(phi3 + epsilon) * phi2

    dist_sq = (u1_dev * range_lat_inv).pow(2) + (u2_dev * range_lon_inv).pow(2)
    distance = torch.sqrt(dist_sq + epsilon) 

    cov_smooth = sigmasq * torch.exp(-distance)

    is_zero_lag = (torch.abs(u1_dev) < 1e-9) & (torch.abs(u2_dev) < 1e-9) & (torch.abs(t_dev) < 1e-9)
    final_cov = torch.where(is_zero_lag, cov_smooth + nugget, cov_smooth)

    if torch.isnan(final_cov).any(): print("Warning: NaN detected in cov_x_spatial_model_kernel output.")
    return final_cov


def cov_spatial_difference(u1, u2, t, params, delta1, delta2):
    """
    Calculates covariance Cov(Y(s), Y(s+u)) for Y(s) = X(s+d1) + X(s+d2) - 2X(s)
    Based on the underlying spatial kernel.
    u1, u2 are PHYSICAL lags.
    """
    weights = {(0, 0): -2.0, (1, 0): 1.0, (0, 1): 1.0}
    device = params.device
    out_shape = torch.broadcast_shapes(u1.shape if isinstance(u1, torch.Tensor) else (),
                                     u2.shape if isinstance(u2, torch.Tensor) else (),
                                     t.shape if isinstance(t, torch.Tensor) else ())
    cov = torch.zeros(out_shape, device=device, dtype=torch.float32)
    u1_dev = u1.to(device) if isinstance(u1, torch.Tensor) else torch.tensor(u1, device=device, dtype=torch.float32)
    u2_dev = u2.to(device) if isinstance(u2, torch.Tensor) else torch.tensor(u2, device=device, dtype=torch.float32)
    t_dev = t.to(device) if isinstance(t, torch.Tensor) else torch.tensor(t, device=device, dtype=torch.float32)

    for (a_idx, b_idx), w_ab in weights.items():
        offset_a1 = a_idx * delta1
        offset_a2 = b_idx * delta2
        for (c_idx, d_idx), w_cd in weights.items():
            offset_c1 = c_idx * delta1
            offset_c2 = d_idx * delta2
            
            # All lags are physical lags
            lag_u1 = u1_dev + (offset_a1 - offset_c1)
            lag_u2 = u2_dev + (offset_a2 - offset_c2)
            
            term_cov = cov_x_spatial_model_kernel(lag_u1, lag_u2, t_dev, params) 
            
            if torch.isnan(term_cov).any():
                 print(f"Warning: NaN in term_cov within cov_spatial_difference.")
                 return torch.full_like(cov, float('nan'))
            cov += w_ab * w_cd * term_cov

    if torch.isnan(cov).any(): print("Warning: NaN in final cov_spatial_difference output.")
    return cov

def cn_bar_tapered(u1, u2, t, params, n1, n2, taper_autocorr_grid, delta1, delta2):
    """
    Computes c_Y(u) * c_gn(u).
    u1, u2 are GRID index lags (e.g., -n1..0..n1)
    """
    device = params.device
    u1_dev = u1.to(device) if isinstance(u1, torch.Tensor) else torch.tensor(u1, device=device, dtype=torch.float32)
    u2_dev = u2.to(device) if isinstance(u2, torch.Tensor) else torch.tensor(u2, device=device, dtype=torch.float32)
    t_dev = t.to(device) if isinstance(t, torch.Tensor) else torch.tensor(t, device=device, dtype=torch.float32)

    # --- Convert GRID lags to PHYSICAL lags ---
    lag_u1 = u1_dev * delta1
    lag_u2 = u2_dev * delta2
    
    # Calculate theoretical covariance c_Y(u) using physical lags
    cov_X_value = cov_spatial_difference(lag_u1, lag_u2, t_dev, params, delta1, delta2)

    # --- Get Taper Autocorrelation Value c_gn(u) from grid ---
    # Use GRID lags for indexing the taper grid
    u1_idx = u1_dev.long()
    u2_idx = u2_dev.long()

    idx1 = (n1 - 1 + u1_idx) # Centering index
    idx2 = (n2 - 1 + u2_idx) # Centering index
    
    idx1 = torch.clamp(idx1, 0, 2 * n1 - 2)
    idx2 = torch.clamp(idx2, 0, 2 * n2 - 2)

    taper_autocorr_value = taper_autocorr_grid[idx1, idx2]

    if torch.isnan(cov_X_value).any() or torch.isnan(taper_autocorr_value).any():
        print("Warning: NaN detected before multiplication in cn_bar_tapered.")
        out_shape = torch.broadcast_shapes(cov_X_value.shape, taper_autocorr_value.shape)
        return torch.full(out_shape, float('nan'), device=device, dtype=torch.float32)

    result = cov_X_value * taper_autocorr_value
    if torch.isnan(result).any(): print("Warning: NaN in cn_bar_tapered output.")
    return result

# --- 💥 RIGOROUSLY REVISED FUNCTION 💥 ---
def expected_periodogram_fft_tapered(params, n1, n2, p, taper_autocorr_grid, delta1, delta2):
    """
    Calculates the expected periodogram I(omega_s) (a pxp matrix in time)
    using the exact taper autocorrelation c_gn(u) and
    CORRECTLY implementing the aliasing sum (Lemma 2).
    """
    device = params.device if isinstance(params, torch.Tensor) else params[0].device
    if isinstance(params, list):
        params_tensor = torch.cat([p.to(device) for p in params])
    else:
        params_tensor = params.to(device)

    # 1. Create the grid for u from [0, n-1]
    # This is the grid for the *output* of the aliased function tilde_c_n(u)
    u1_lags = torch.arange(n1, dtype=torch.float32, device=device)
    u2_lags = torch.arange(n2, dtype=torch.float32, device=device)
    u1_mesh, u2_mesh = torch.meshgrid(u1_lags, u2_lags, indexing='ij')

    t_lags = torch.arange(p, dtype=torch.float32, device=device)
    tilde_cn_tensor = torch.zeros((n1, n2, p, p), dtype=torch.complex64, device=device)

    # 2. Loop over the p x p matrix (for this spatial code, p=1)
    for q in range(p):
        for r in range(p):
            t_diff = t_lags[q] - t_lags[r]
            
            # --- Calculate 4 aliasing terms from Lemma 2 ---
            # cn_bar_tapered handles any GRID lag (positive or negative)
            
            # Term 1: c_n(u1, u2, t_diff)
            term1 = cn_bar_tapered(u1_mesh, u2_mesh, t_diff, 
                                   params_tensor, n1, n2, taper_autocorr_grid, delta1, delta2)

            # Term 2: c_n(u1 - n1, u2, t_diff)
            term2 = cn_bar_tapered(u1_mesh - n1, u2_mesh, t_diff, 
                                   params_tensor, n1, n2, taper_autocorr_grid, delta1, delta2)

            # Term 3: c_n(u1, u2 - n2, t_diff)
            term3 = cn_bar_tapered(u1_mesh, u2_mesh - n2, t_diff, 
                                   params_tensor, n1, n2, taper_autocorr_grid, delta1, delta2)

            # Term 4: c_n(u1 - n1, u2 - n2, t_diff)
            term4 = cn_bar_tapered(u1_mesh - n1, u2_mesh - n2, t_diff,
                                   params_tensor, n1, n2, taper_autocorr_grid, delta1, delta2)
            
            # 3. Sum them to create tilde_c_n(u, t_diff)
            tilde_cn_grid_qr = (term1 + term2 + term3 + term4)
            
            if torch.isnan(tilde_cn_grid_qr).any():
                 tilde_cn_tensor[:, :, q, r] = float('nan')
            else:
                 tilde_cn_tensor[:, :, q, r] = tilde_cn_grid_qr.to(torch.complex64)

    if torch.isnan(tilde_cn_tensor).any():
        print("Warning: NaN detected in tilde_cn_tensor before FFT.")
        nan_shape = (n1, n2, p, p)
        return torch.full(nan_shape, float('nan'), dtype=torch.complex64, device=device)

    # 4. Compute FFT of the *aliased* covariance
    fft_result = torch.fft.fft2(tilde_cn_tensor, dim=(0, 1))
    
    # Result should be real (up to numerical noise)
    fft_result_real = fft_result.real 
    
    normalization_factor = 1.0 / (4.0 * cmath.pi**2)
    result = fft_result_real * normalization_factor

    if torch.isnan(result).any(): print("Warning: NaN in expected_periodogram_fft_tapered output.")
    return result
# --- 💥 END REVISION 💥 ---


# =========================================================================
# 3. Data Processing (MODIFIED for HAMMING TAPERING)
# =========================================================================
def generate_Jvector_tapered(tensor_list, tapering_func, lat_col, lon_col, val_col, device):
    """
    Generates J-vector for a single component using the specified taper,
    placing result on device.
    """
    p = len(tensor_list)
    if p == 0: return torch.empty(0, 0, 0, device=device), 0, 0, 0, None 

    valid_tensors = [t for t in tensor_list if t.numel() > 0 and t.shape[1] > max(lat_col, lon_col, val_col)]
    if not valid_tensors:
         print("Warning: No valid tensors found in tensor_list.")
         return torch.empty(0, 0, 0, device=device), 0, 0, 0, None

    try:
        all_lats_cpu = torch.cat([t[:, lat_col] for t in valid_tensors])
        all_lons_cpu = torch.cat([t[:, lon_col] for t in valid_tensors])
    except IndexError:
        print(f"Error: Invalid column index. Check tensor shapes.")
        return torch.empty(0, 0, 0, device=device), 0, 0, 0, None

    all_lats_cpu = all_lats_cpu[~torch.isnan(all_lats_cpu) & ~torch.isinf(all_lats_cpu)]
    all_lons_cpu = all_lons_cpu[~torch.isnan(all_lons_cpu) & ~torch.isinf(all_lons_cpu)]
    if all_lats_cpu.numel() == 0 or all_lons_cpu.numel() == 0:
        print("Warning: No valid coordinates after NaN/Inf filtering.")
        return torch.empty(0, 0, 0, device=device), 0, 0, 0, None

    unique_lats_cpu, unique_lons_cpu = torch.unique(all_lats_cpu), torch.unique(all_lons_cpu)
    n1, n2 = len(unique_lats_cpu), len(unique_lons_cpu)
    if n1 == 0 or n2 == 0:
        print("Warning: Grid dimensions are zero.")
        return torch.empty(0, 0, 0, device=device), 0, 0, 0, None

    lat_map = {lat.item(): i for i, lat in enumerate(unique_lats_cpu)}
    lon_map = {lon.item(): i for i, lon in enumerate(unique_lons_cpu)}

    # --- Create Taper Grid ---
    u1_mesh_cpu, u2_mesh_cpu = torch.meshgrid(
        torch.arange(n1, dtype=torch.float32),
        torch.arange(n2, dtype=torch.float32),
        indexing='ij'
    )
    taper_grid = tapering_func((u1_mesh_cpu, u2_mesh_cpu), n1, n2).to(device) # Taper on device

    fft_results = []
    for tensor in tensor_list:
        data_grid = torch.zeros((n1, n2), dtype=torch.float32, device=device)
        if tensor.numel() > 0 and tensor.shape[1] > max(lat_col, lon_col, val_col):
            for row in tensor:
                lat_item, lon_item = row[lat_col].item(), row[lon_col].item()
                if not (np.isnan(lat_item) or np.isnan(lon_item)):
                    i = lat_map.get(lat_item)
                    j = lon_map.get(lon_item)
                    if i is not None and j is not None:
                        val = row[val_col]
                        val_num = val.item() if isinstance(val, torch.Tensor) else val
                        if not np.isnan(val_num) and not np.isinf(val_num):
                           data_grid[i, j] = val_num

        data_grid_tapered = data_grid * taper_grid 

        if torch.isnan(data_grid_tapered).any() or torch.isinf(data_grid_tapered).any():
             print("Warning: NaN/Inf detected in data_grid_tapered before FFT. Replacing with zeros.")
             data_grid_tapered = torch.nan_to_num(data_grid_tapered, nan=0.0, posinf=0.0, neginf=0.0)

        fft_results.append(torch.fft.fft2(data_grid_tapered))

    if not fft_results:
         print("Warning: No FFT results generated.")
         return torch.empty(0, 0, 0, device=device), n1, n2, 0, taper_grid

    J_vector_tensor = torch.stack(fft_results, dim=2).to(device)

    H = torch.sum(taper_grid**2)
    if H < 1e-12:
        print("Warning: Normalization factor H is near zero.")
        norm_factor = torch.tensor(0.0, device=device)
    else:
        norm_factor = (torch.sqrt(1.0 / H) / (2.0 * cmath.pi)).to(device)

    result = J_vector_tensor * norm_factor
    if torch.isnan(result).any(): print("Warning: NaN in J_vector output.")
    return result, n1, n2, p, taper_grid


def calculate_sample_periodogram_vectorized(J_vector_tensor):
    """Calculates sample periodogram I_n = J J^H (pxp matrix for each spatial freq)."""
    if torch.isnan(J_vector_tensor).any() or torch.isinf(J_vector_tensor).any():
        print("Warning: NaN/Inf detected in J_vector_tensor input.")
        n1, n2, p = J_vector_tensor.shape
        return torch.full((n1, n2, p, p), float('nan'), dtype=torch.complex64, device=J_vector_tensor.device)

    J_col = J_vector_tensor.unsqueeze(-1)
    J_row_conj = J_vector_tensor.unsqueeze(-2).conj()
    result = J_col @ J_row_conj

    if torch.isnan(result).any(): print("Warning: NaN in periodogram matrix output.")
    return result


# =========================================================================
# 4. Likelihood Calculation (Adapted for Tapering with Autocorrelation)
# =========================================================================

def whittle_likelihood_loss_tapered(params, I_sample, n1, n2, p, taper_autocorr_grid, delta1, delta2):
    """
    ✅ Whittle Likelihood Loss using data tapering and exact taper autocorrelation c_gn.
    Models a single field (the spatially differenced one). 
    Uses SpatialModel kernel.
    """
    device = I_sample.device
    params_tensor = params.to(device)

    if torch.isnan(params_tensor).any() or torch.isinf(params_tensor).any():
        print("Warning: NaN/Inf detected in input parameters to likelihood.")
        return torch.tensor(float('nan'), device=device)

    # --- 💥 MODIFIED: Call the corrected function 💥 ---
    # The 'cov_func' argument is removed as it's no longer needed.
    I_expected = expected_periodogram_fft_tapered(
        params_tensor, n1, n2, p, taper_autocorr_grid, 
        delta1, delta2
    )
    # --- END MODIFICATION ---

    if torch.isnan(I_expected).any() or torch.isinf(I_expected).any():
        print("Warning: NaN/Inf returned from expected_periodogram calculation.")
        return torch.tensor(float('nan'), device=device)

    eye_matrix = torch.eye(p, dtype=torch.complex64, device=device)
    diag_vals = torch.abs(I_expected.diagonal(dim1=-2, dim2=-1))
    mean_diag_abs = diag_vals.mean().item() if diag_vals.numel() > 0 and not torch.isnan(diag_vals).all() else 1.0
    diag_load = max(mean_diag_abs * 1e-8, 1e-9)
    I_expected_stable = I_expected + eye_matrix * diag_load

    sign, logabsdet = torch.linalg.slogdet(I_expected_stable)
    if torch.any(sign.real <= 1e-9):
        print("Warning: Non-positive determinant encountered. Applying penalty.")
        log_det_term = torch.where(sign.real > 1e-9, logabsdet, torch.tensor(1e10, device=device))
    else:
        log_det_term = logabsdet

    if torch.isnan(I_sample).any() or torch.isinf(I_sample).any():
        print("Warning: NaN/Inf detected in I_sample input to likelihood.")
        return torch.tensor(float('nan'), device=device)

    try:
        solved_term = torch.linalg.solve(I_expected_stable, I_sample)
        trace_term = torch.einsum('...ii->...', solved_term).real
    except torch.linalg.LinAlgError as e:
        print(f"Warning: LinAlgError during solve: {e}. Applying high loss penalty.")
        return torch.tensor(float('inf'), device=device)

    if torch.isnan(trace_term).any() or torch.isinf(trace_term).any():
        print("Warning: NaN/Inf detected in trace_term. Returning NaN loss.")
        return torch.tensor(float('nan'), device=device)

    likelihood_terms = log_det_term + trace_term

    if torch.isnan(likelihood_terms).any():
        print("Warning: NaN detected in likelihood_terms before summation. Returning NaN loss.")
        return torch.tensor(float('nan'), device=device)

    total_sum = torch.sum(likelihood_terms)
    dc_term = likelihood_terms[0, 0] if n1 > 0 and n2 > 0 else torch.tensor(0.0, device=device)
    if torch.isnan(dc_term).any() or torch.isinf(dc_term).any():
        print("Warning: NaN/Inf detected in DC term. Setting to 0.")
        dc_term = torch.tensor(0.0, device=device)

    loss = total_sum - dc_term if (n1 > 1 or n2 > 1) else total_sum

    if torch.isnan(loss) or torch.isinf(loss):
         print("Warning: NaN/Inf detected in final loss. Returning Inf penalty.")
         return torch.tensor(float('inf'), device=device)

    return loss


# =========================================================================
# 5. Training Loop & Helpers (💥 MODIFIED for 2-style param printing 💥)
# =========================================================================

def get_printable_params(p_list):
    """Helper to convert log-params to natural scale for printing."""
    valid_tensors = [p for p in p_list if isinstance(p, torch.Tensor)]
    if not valid_tensors: return "Invalid params_list"
    p_cat = torch.cat([p.detach().clone().cpu() for p in valid_tensors])
    
    if len(p_cat) != 4:
        return f"Expected 4 params, got {len(p_cat)}. Data: {p_cat.numpy().round(4)}"

    log_params = p_cat
    if torch.isnan(log_params).any() or torch.isinf(log_params).any():
        return "[NaN/Inf in log_params]"
        
    try:
        phi1 = torch.exp(log_params[0])
        phi2 = torch.exp(log_params[1]) # range_lon_inv
        phi3 = torch.exp(log_params[2]) # (range_lon / range_lat)^2
        nugget = torch.exp(log_params[3])

        epsilon = 1e-12
        sigmasq = phi1 / (phi2 + epsilon)
        range_lon = 1.0 / (phi2 + epsilon)
        range_lat = 1.0 / (torch.sqrt(phi3 + epsilon) * phi2 + epsilon)
        
        return (f"sigmasq: {sigmasq.item():.4f}, range_lat: {range_lat.item():.4f}, "
                f"range_lon: {range_lon.item():.4f}, nugget: {nugget.item():.4f}")
    except Exception as e:
        return f"[Error in param conversion: {e}]"

# --- 💥 NEW HELPER 💥 ---
def get_phi_params(log_params_list):
    """Helper to print reparameterized (phi-scale) params."""
    try:
        p_cat = torch.cat([p.detach().clone().cpu() for p in log_params_list])
        phi1 = torch.exp(p_cat[0])
        phi2 = torch.exp(p_cat[1])
        phi3 = torch.exp(p_cat[2])
        nugget = torch.exp(p_cat[3]) 
        
        return (f"phi1: {phi1.item():.4f}, phi2: {phi2.item():.4f}, "
                f"phi3: {phi3.item():.4f}, nugget: {nugget.item():.4f}")
    except Exception:
        return "[Error in reparam conversion]"
# --- END NEW HELPER ---


def run_full_tapered(params_list, optimizer, scheduler, I_sample, n1, n2, p, taper_autocorr_grid, epochs=600, device='cpu'):
    """Corrected training loop using 4-parameter list and tapered likelihood."""
    best_loss = float('inf')
    params_list = [p.to(device) for p in params_list]
    best_params_state = [p.detach().clone() for p in params_list]
    epochs_completed = 0
    DELTA_LAT, DELTA_LON = 0.044, 0.063 # Grid spacing needed for cov func

    I_sample_dev = I_sample.to(device)
    taper_autocorr_grid_dev = taper_autocorr_grid.to(device) 

    for epoch in range(epochs):
        epochs_completed = epoch + 1
        optimizer.zero_grad()
        params_tensor = torch.cat(params_list) 

        loss = whittle_likelihood_loss_tapered(
            params_tensor, I_sample_dev, n1, n2, p, taper_autocorr_grid_dev, DELTA_LAT, DELTA_LON
        )

        if torch.isnan(loss) or torch.isinf(loss):
            print(f"Loss became NaN or Inf at epoch {epoch+1}. Stopping.")
            if epoch == 0: best_params_state = None
            epochs_completed = epoch
            break

        loss.backward()

        nan_grad = False
        for param in params_list:
            if param.grad is not None and (torch.isnan(param.grad).any() or torch.isinf(param.grad).any()):
                nan_grad = True; break
        if nan_grad:
             print(f"Warning: NaN/Inf gradient at epoch {epoch+1}. Skipping step.")
             optimizer.zero_grad()
             continue

        all_params_on_device = params_list
        if all_params_on_device:
            torch.nn.utils.clip_grad_norm_(all_params_on_device, max_norm=1.0)

        optimizer.step()
        scheduler.step()

        current_loss_item = loss.item()
        if current_loss_item < best_loss:
            params_valid = not any(torch.isnan(p.data).any() or torch.isinf(p.data).any() for p in params_list)
            if params_valid:
                best_loss = current_loss_item
                best_params_state = [p.detach().clone() for p in params_list]
            else:
                print(f"Warning: NaN/Inf in params after step epoch {epoch+1}. Not saving.")

        current_lr = optimizer.param_groups[0]['lr'] if optimizer.param_groups else 0.0

        if epoch % 50 == 0 or epoch == epochs - 1:
            print(f'--- Epoch {epoch+1}/{epochs} (LR: {current_lr:.6f}) ---')
            print(f' Loss: {current_loss_item:.4f}')
            print(f' Parameters (Natural Scale): {get_printable_params(params_list)}')


    print("\n--- Training Complete ---")
    if best_params_state is None:
        print("Training failed to find a valid model state.")
        return None, None, epochs_completed # Return None for params

    final_params_log_scale = torch.cat([p.cpu() for p in best_params_state])
    
    # Get final params in both formats
    final_natural_params_str = get_printable_params(best_params_state)
    final_phi_params_str = get_phi_params(best_params_state)
    final_loss_rounded = round(best_loss, 3) if best_loss != float('inf') else float('inf')

    print(f'\nFINAL BEST STATE ACHIEVED (during training):')
    print(f'Best Loss: {final_loss_rounded}')
    print(f'Parameters (Natural Scale): {final_natural_params_str}')
    print(f'Parameters (Phi Scale)    : {final_phi_params_str}')
    
    # Return strings for final printing
    return final_natural_params_str, final_phi_params_str, final_loss_rounded, epochs_completed


# =========================================================================
# 6. Main Execution Script (💥 MODIFIED 💥)
# =========================================================================
if __name__ == '__main__':
    start_time = time.time()

    # --- Configuration ---
    DAY_TO_RUN = 1
    TAPERING_FUNC = cgn_hamming # Use Hamming taper
    NUM_RUNS = 1
    EPOCHS = 5000 
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {DEVICE}")

    DELTA_LAT, DELTA_LON = 0.044, 0.063 

    LAT_COL, LON_COL = 0, 1
    VAL_COL = 2 # Spatially differenced value
    TIME_COL = 3

    # --- Load Spatially Differenced Data ---
    try:
        with open("spatial_first_difference_data.pkl", 'rb') as f:
            processed_df = pickle.load(f)
        print(f"Loaded {len(processed_df)} days from spatial_first_difference_data.pkl.")
        processed_df = [
            torch.tensor(arr, dtype=torch.float32).cpu() if not isinstance(arr, torch.Tensor)
            else arr.cpu().to(torch.float32)
            for arr in processed_df
        ]
        if not processed_df: raise ValueError("'processed_df' is empty.")
    except FileNotFoundError:
        print("Error: `spatial_first_difference_data.pkl` not found.")
        exit()
    except Exception as e:
        print(f"Error loading or processing 'processed_df': {e}")
        exit()

    if DAY_TO_RUN > len(processed_df) or DAY_TO_RUN <= 0:
        print(f"Error: DAY_TO_RUN ({DAY_TO_RUN}) out of bounds.")
        exit()

    cur_df = processed_df[DAY_TO_RUN - 1]
    
    # --- Filter to a single time point (p=1) ---
    cur_df = cur_df[cur_df[:,3] == cur_df[:,3].min()]
    print(f"Filtered data to single time point. New shape: {cur_df.shape}")
    
    if cur_df.numel() == 0 or cur_df.shape[1] <= max(LAT_COL, LON_COL, VAL_COL, TIME_COL):
        print(f"Error: Data for Day {DAY_TO_RUN} (single time point) is empty or invalid.")
        exit()

    time_slices_list = [cur_df] # List containing the single time slice
    
    # --- 1. Pre-compute J-vector, Taper Grid, and Taper Autocorrelation ---
    print("Pre-computing J-vector (Hamming taper)...")
    J_vec, n1, n2, p, taper_grid = generate_Jvector_tapered( 
        time_slices_list,
        tapering_func=TAPERING_FUNC, 
        lat_col=LAT_COL, lon_col=LON_COL, val_col=VAL_COL,
        device=DEVICE
    )

    if J_vec is None or J_vec.numel() == 0 or n1 == 0 or n2 == 0 or p == 0:
       print(f"Error: J-vector generation failed for Day {DAY_TO_RUN}.")
       exit()
       
    print("Pre-computing sample periodogram...")
    I_sample = calculate_sample_periodogram_vectorized(J_vec)

    print("Pre-computing Hamming taper autocorrelation...")
    taper_autocorr_grid = calculate_taper_autocorrelation_fft(taper_grid, n1, n2, DEVICE)

    if torch.isnan(I_sample).any() or torch.isinf(I_sample).any():
        print("Error: NaN/Inf in sample periodogram.")
        exit()
    if torch.isnan(taper_autocorr_grid).any() or torch.isinf(taper_autocorr_grid).any():
        print("Error: NaN/Inf in taper autocorrelation.")
        exit()

    print(f"Data grid: {n1}x{n2}, {p} time points. J-vector, Periodogram, Taper Autocorr on {DEVICE}.")

    # --- 2. Optimization Loop ---
    all_final_results = []
    all_final_losses = []

    for i in range(NUM_RUNS):
        print(f"\n{'='*30} Initialization Run {i+1}/{NUM_RUNS} {'='*30}")

        # [log_phi1, log_phi2, log_phi3, log_nugget]
        initial_params_values = [np.log(15.0), np.log(0.66), np.log(0.7), np.log(1.5)]
        
        print(f"Starting with FIXED params (log-scale): {[round(p, 4) for p in initial_params_values]}")

        params_list = [
            Parameter(torch.tensor([val], dtype=torch.float32))
            for val in initial_params_values
        ]

        lr = 0.01 
        optimizer = torch.optim.Adam(params_list, lr=lr)

        T_MAX = 200
        ETA_MIN = 1e-6
        scheduler = CosineAnnealingLR(optimizer, T_max=T_MAX, eta_min=ETA_MIN)

        print(f"Starting optimization run {i+1} on device {DEVICE} (Hamming, SpatialModel kernel)...")
        
        nat_params_str, phi_params_str, loss, epochs_run = run_full_tapered(
            params_list=params_list,
            optimizer=optimizer,
            scheduler=scheduler,
            I_sample=I_sample,
            n1=n1, n2=n2, p=p,
            taper_autocorr_grid=taper_autocorr_grid, 
            epochs=EPOCHS,
            device=DEVICE
        )
        
        # Store results for final summary
        if loss is not None:
            all_final_results.append((nat_params_str, phi_params_str))
            all_final_losses.append(loss)
        else:
            all_final_losses.append(float('inf'))

    print(f"\n\n{'='*25} Overall Result from Run {'='*25}")
    valid_losses = [l for l in all_final_losses if l is not None and l != float('inf')]

    if not valid_losses:
        print(f"The run failed or resulted in an invalid loss for Day {DAY_TO_RUN}.")
    else:
        best_loss = min(valid_losses)
        best_run_index = all_final_losses.index(best_loss)
        best_results = all_final_results[best_run_index]
        
        print(f"Best Run Loss: {best_loss}")
        print(f"Final Parameters (Natural Scale): {best_results[0]}")
        print(f"Final Parameters (Phi Scale)    : {best_results[1]}")

    end_time = time.time()
    print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")

Using device: cpu
Loaded 31 days from spatial_first_difference_data.pkl.
Filtered data to single time point. New shape: torch.Size([17854, 4])
Pre-computing J-vector (Hamming taper)...
Pre-computing sample periodogram...
Pre-computing Hamming taper autocorrelation...
Data grid: 113x158, 1 time points. J-vector, Periodogram, Taper Autocorr on cpu.

============================== Initialization Run 1/1 ==============================
Starting with FIXED params (log-scale): [2.7081, -0.4155, -0.3567, 0.4055]
Starting optimization run 1 on device cpu (Hamming, SpatialModel kernel)...
--- Epoch 1/5000 (LR: 0.009999) ---
 Loss: 8893.6602
 Parameters (Natural Scale): sigmasq: 23.1864, range_lat: 1.8200, range_lon: 1.5304, nugget: 1.5151
--- Epoch 51/5000 (LR: 0.008480) ---
 Loss: 4808.0454
 Parameters (Natural Scale): sigmasq: 54.4521, range_lat: 2.0226, range_lon: 2.1998, nugget: 2.3851
--- Epoch 101/5000 (LR: 0.004922) ---
 Loss: 4508.7007
 Parameters (Natural Scale): sigmasq: 38.2246, range

In [35]:
import torch
import numpy as np
import matplotlib.pyplot as plt 
import cmath
import pickle
import time 
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.nn import Parameter
import torch.fft 
from typing import List, Dict, Any, Callable

# =========================================================================
# 1. Tapering, Autocorrelation, and Covariance Functions
# =========================================================================

def cgn_hamming(u, n1, n2):
    """Computes a 2D Hamming window."""
    u1, u2 = u
    device = u1.device if isinstance(u1, torch.Tensor) else (u2.device if isinstance(u2, torch.Tensor) else torch.device('cpu'))
    u1_tensor = u1.to(device) if isinstance(u1, torch.Tensor) else torch.tensor(u1, device=device, dtype=torch.float32)
    u2_tensor = u2.to(device) if isinstance(u2, torch.Tensor) else torch.tensor(u2, device=device, dtype=torch.float32)

    n1_eff = float(n1) if n1 > 0 else 1.0
    n2_eff = float(n2) if n2 > 0 else 1.0
    hamming1 = 0.54 + 0.46 * torch.cos(2.0 * torch.pi * u1_tensor / n1_eff)
    hamming2 = 0.54 + 0.46 * torch.cos(2.0 * torch.pi * u2_tensor / n2_eff)
    return hamming1 * hamming2

def cgn_2dbartlett(u, n1, n2): # Kept for potential future use
    """Computes a 2D Bartlett window function."""
    u1, u2 = u
    device = u1.device if isinstance(u1, torch.Tensor) else (u2.device if isinstance(u2, torch.Tensor) else torch.device('cpu'))
    u1_tensor = u1.to(device) if isinstance(u1, torch.Tensor) else torch.tensor(u1, device=device, dtype=torch.float32)
    u2_tensor = u2.to(device) if isinstance(u2, torch.Tensor) else torch.tensor(u2, device=device, dtype=torch.float32)
    n1_eff = float(n1) if n1 > 0 else 1.0
    n2_eff = float(n2) if n2 > 0 else 1.0
    kernel = (1.0 - torch.abs(u1_tensor) / n1_eff) * (1.0 - torch.abs(u2_tensor) / n2_eff)
    return torch.clamp(kernel, min=0.0)

def calculate_taper_autocorrelation_fft(taper_grid, n1, n2, device):
    """
    Computes the normalized taper autocorrelation function c_gn(u) using FFT.
    """
    taper_grid = taper_grid.to(device) 
    H = torch.sum(taper_grid**2)
    if H < 1e-12:
        print("Warning: Sum of squared taper weights (H) is near zero.")
        return torch.zeros((2*n1-1, 2*n2-1), device=device, dtype=taper_grid.dtype)

    N1, N2 = 2 * n1 - 1, 2 * n2 - 1
    taper_fft = torch.fft.fft2(taper_grid, s=(N1, N2))
    power_spectrum = torch.abs(taper_fft)**2
    autocorr_unnormalized = torch.fft.ifft2(power_spectrum).real
    autocorr_shifted = torch.fft.fftshift(autocorr_unnormalized)

    c_gn_grid = autocorr_shifted / (H + 1e-12)

    return c_gn_grid 

def cov_x_spatial_model_kernel(u1, u2, t, params):
    """
    Computes autocovariance of X using the SPATIAL-ONLY kernel.
    u1, u2 are PHYSICAL lags (already scaled by deltas).
    """
    device = params.device
    u1_dev = u1.to(device) if isinstance(u1, torch.Tensor) else torch.tensor(u1, device=device, dtype=torch.float32)
    u2_dev = u2.to(device) if isinstance(u2, torch.Tensor) else torch.tensor(u2, device=device, dtype=torch.float32)
    t_dev = t.to(device) if isinstance(t, torch.Tensor) else torch.tensor(t, device=device, dtype=torch.float32)

    if torch.isnan(params).any() or torch.isinf(params).any():
         out_shape = torch.broadcast_shapes(u1_dev.shape, u2_dev.shape, t_dev.shape)
         return torch.full(out_shape, float('nan'), device=device, dtype=torch.float32)

    phi1   = torch.exp(params[0])
    phi2   = torch.exp(params[1]) 
    phi3   = torch.exp(params[2]) 
    nugget = torch.exp(params[3])

    epsilon = 1e-12
    sigmasq = phi1 / (phi2 + epsilon)  
    range_lon_inv = phi2
    range_lat_inv = torch.sqrt(phi3 + epsilon) * phi2

    dist_sq = (u1_dev * range_lat_inv).pow(2) + (u2_dev * range_lon_inv).pow(2)
    distance = torch.sqrt(dist_sq + epsilon) 

    cov_smooth = sigmasq * torch.exp(-distance)

    is_zero_lag = (torch.abs(u1_dev) < 1e-9) & (torch.abs(u2_dev) < 1e-9) & (torch.abs(t_dev) < 1e-9)
    final_cov = torch.where(is_zero_lag, cov_smooth + nugget, cov_smooth)

    if torch.isnan(final_cov).any(): print("Warning: NaN detected in cov_x_spatial_model_kernel output.")
    return final_cov


def cov_spatial_difference(u1, u2, t, params, delta1, delta2):
    """
    Calculates covariance Cov(Y(s), Y(s+u)) for Y(s) = X(s+d1) + X(s+d2) - 2X(s)
    Based on the underlying spatial kernel.
    u1, u2 are PHYSICAL lags.
    """
    weights = {(0, 0): -2.0, (1, 0): 1.0, (0, 1): 1.0}
    device = params.device
    out_shape = torch.broadcast_shapes(u1.shape if isinstance(u1, torch.Tensor) else (),
                                     u2.shape if isinstance(u2, torch.Tensor) else (),
                                     t.shape if isinstance(t, torch.Tensor) else ())
    cov = torch.zeros(out_shape, device=device, dtype=torch.float32)
    u1_dev = u1.to(device) if isinstance(u1, torch.Tensor) else torch.tensor(u1, device=device, dtype=torch.float32)
    u2_dev = u2.to(device) if isinstance(u2, torch.Tensor) else torch.tensor(u2, device=device, dtype=torch.float32)
    t_dev = t.to(device) if isinstance(t, torch.Tensor) else torch.tensor(t, device=device, dtype=torch.float32)

    for (a_idx, b_idx), w_ab in weights.items():
        offset_a1 = a_idx * delta1
        offset_a2 = b_idx * delta2
        for (c_idx, d_idx), w_cd in weights.items():
            offset_c1 = c_idx * delta1
            offset_c2 = d_idx * delta2
            
            lag_u1 = u1_dev + (offset_a1 - offset_c1)
            lag_u2 = u2_dev + (offset_a2 - offset_c2)
            
            term_cov = cov_x_spatial_model_kernel(lag_u1, lag_u2, t_dev, params) 
            
            if torch.isnan(term_cov).any():
                 print(f"Warning: NaN in term_cov within cov_spatial_difference.")
                 return torch.full_like(cov, float('nan'))
            cov += w_ab * w_cd * term_cov

    if torch.isnan(cov).any(): print("Warning: NaN in final cov_spatial_difference output.")
    return cov

def cn_bar_tapered(u1, u2, t, params, n1, n2, taper_autocorr_grid, delta1, delta2):
    """
    Computes c_Y(u) * c_gn(u).
    u1, u2 are GRID index lags (e.g., -n1..0..n1)
    """
    device = params.device
    u1_dev = u1.to(device) if isinstance(u1, torch.Tensor) else torch.tensor(u1, device=device, dtype=torch.float32)
    u2_dev = u2.to(device) if isinstance(u2, torch.Tensor) else torch.tensor(u2, device=device, dtype=torch.float32)
    t_dev = t.to(device) if isinstance(t, torch.Tensor) else torch.tensor(t, device=device, dtype=torch.float32)

    # --- Convert GRID lags to PHYSICAL lags ---
    lag_u1 = u1_dev * delta1
    lag_u2 = u2_dev * delta2
    
    cov_X_value = cov_spatial_difference(lag_u1, lag_u2, t_dev, params, delta1, delta2)

    # --- Get Taper Autocorrelation Value c_gn(u) from grid ---
    u1_idx = u1_dev.long()
    u2_idx = u2_dev.long()

    idx1 = (n1 - 1 + u1_idx) # Centering index
    idx2 = (n2 - 1 + u2_idx) # Centering index
    
    idx1 = torch.clamp(idx1, 0, 2 * n1 - 2)
    idx2 = torch.clamp(idx2, 0, 2 * n2 - 2)

    taper_autocorr_value = taper_autocorr_grid[idx1, idx2]

    if torch.isnan(cov_X_value).any() or torch.isnan(taper_autocorr_value).any():
        out_shape = torch.broadcast_shapes(cov_X_value.shape, taper_autocorr_value.shape)
        return torch.full(out_shape, float('nan'), device=device, dtype=torch.float32)

    result = cov_X_value * taper_autocorr_value
    if torch.isnan(result).any(): print("Warning: NaN in cn_bar_tapered output.")
    return result


def expected_periodogram_fft_tapered(params, n1, n2, p, taper_autocorr_grid, delta1, delta2):
    """
    Calculates the expected periodogram I(omega_s) (a pxp matrix in time)
    using the exact taper autocorrelation c_gn(u) and
    CORRECTLY implementing the aliasing sum (Lemma 2).
    """
    device = params.device if isinstance(params, torch.Tensor) else params[0].device
    if isinstance(params, list):
        params_tensor = torch.cat([p.to(device) for p in params])
    else:
        params_tensor = params.to(device)

    u1_lags = torch.arange(n1, dtype=torch.float32, device=device)
    u2_lags = torch.arange(n2, dtype=torch.float32, device=device)
    u1_mesh, u2_mesh = torch.meshgrid(u1_lags, u2_lags, indexing='ij')

    t_lags = torch.arange(p, dtype=torch.float32, device=device)
    tilde_cn_tensor = torch.zeros((n1, n2, p, p), dtype=torch.complex64, device=device)

    for q in range(p):
        for r in range(p):
            t_diff = t_lags[q] - t_lags[r]
            
            term1 = cn_bar_tapered(u1_mesh, u2_mesh, t_diff, 
                                   params_tensor, n1, n2, taper_autocorr_grid, delta1, delta2)
            term2 = cn_bar_tapered(u1_mesh - n1, u2_mesh, t_diff, 
                                   params_tensor, n1, n2, taper_autocorr_grid, delta1, delta2)
            term3 = cn_bar_tapered(u1_mesh, u2_mesh - n2, t_diff, 
                                   params_tensor, n1, n2, taper_autocorr_grid, delta1, delta2)
            term4 = cn_bar_tapered(u1_mesh - n1, u2_mesh - n2, t_diff,
                                   params_tensor, n1, n2, taper_autocorr_grid, delta1, delta2)
            
            tilde_cn_grid_qr = (term1 + term2 + term3 + term4)
            
            if torch.isnan(tilde_cn_grid_qr).any():
                 tilde_cn_tensor[:, :, q, r] = float('nan')
            else:
                 tilde_cn_tensor[:, :, q, r] = tilde_cn_grid_qr.to(torch.complex64)

    if torch.isnan(tilde_cn_tensor).any():
        print("Warning: NaN detected in tilde_cn_tensor before FFT.")
        nan_shape = (n1, n2, p, p)
        return torch.full(nan_shape, float('nan'), dtype=torch.complex64, device=device)

    fft_result = torch.fft.fft2(tilde_cn_tensor, dim=(0, 1))
    fft_result_real = fft_result.real 
    normalization_factor = 1.0 / (4.0 * cmath.pi**2)
    result = fft_result_real * normalization_factor

    if torch.isnan(result).any(): print("Warning: NaN in expected_periodogram_fft_tapered output.")
    return result

# =========================================================================
# 3. Data Processing (Tapering)
# =========================================================================
def generate_Jvector_tapered(tensor_list, tapering_func, lat_col, lon_col, val_col, device):
    """
    Generates J-vector for a single component using the specified taper,
    placing result on device.
    """
    p = len(tensor_list)
    if p == 0: return torch.empty(0, 0, 0, device=device), 0, 0, 0, None 

    valid_tensors = [t for t in tensor_list if t.numel() > 0 and t.shape[1] > max(lat_col, lon_col, val_col)]
    if not valid_tensors:
         print("Warning: No valid tensors found in tensor_list.")
         return torch.empty(0, 0, 0, device=device), 0, 0, 0, None

    try:
        all_lats_cpu = torch.cat([t[:, lat_col] for t in valid_tensors])
        all_lons_cpu = torch.cat([t[:, lon_col] for t in valid_tensors])
    except IndexError:
        print(f"Error: Invalid column index. Check tensor shapes.")
        return torch.empty(0, 0, 0, device=device), 0, 0, 0, None

    all_lats_cpu = all_lats_cpu[~torch.isnan(all_lats_cpu) & ~torch.isinf(all_lats_cpu)]
    all_lons_cpu = all_lons_cpu[~torch.isnan(all_lons_cpu) & ~torch.isinf(all_lons_cpu)]
    if all_lats_cpu.numel() == 0 or all_lons_cpu.numel() == 0:
        print("Warning: No valid coordinates after NaN/Inf filtering.")
        return torch.empty(0, 0, 0, device=device), 0, 0, 0, None

    unique_lats_cpu, unique_lons_cpu = torch.unique(all_lats_cpu), torch.unique(all_lons_cpu)
    n1, n2 = len(unique_lats_cpu), len(unique_lons_cpu)
    if n1 == 0 or n2 == 0:
        print("Warning: Grid dimensions are zero.")
        return torch.empty(0, 0, 0, device=device), 0, 0, 0, None

    lat_map = {lat.item(): i for i, lat in enumerate(unique_lats_cpu)}
    lon_map = {lon.item(): i for i, lon in enumerate(unique_lons_cpu)}

    u1_mesh_cpu, u2_mesh_cpu = torch.meshgrid(
        torch.arange(n1, dtype=torch.float32),
        torch.arange(n2, dtype=torch.float32),
        indexing='ij'
    )
    taper_grid = tapering_func((u1_mesh_cpu, u2_mesh_cpu), n1, n2).to(device) # Taper on device

    fft_results = []
    for tensor in tensor_list:
        data_grid = torch.zeros((n1, n2), dtype=torch.float32, device=device)
        if tensor.numel() > 0 and tensor.shape[1] > max(lat_col, lon_col, val_col):
            for row in tensor:
                lat_item, lon_item = row[lat_col].item(), row[lon_col].item()
                if not (np.isnan(lat_item) or np.isnan(lon_item)):
                    i = lat_map.get(lat_item)
                    j = lon_map.get(lon_item)
                    if i is not None and j is not None:
                        val = row[val_col]
                        val_num = val.item() if isinstance(val, torch.Tensor) else val
                        if not np.isnan(val_num) and not np.isinf(val_num):
                           data_grid[i, j] = val_num

        data_grid_tapered = data_grid * taper_grid 

        if torch.isnan(data_grid_tapered).any() or torch.isinf(data_grid_tapered).any():
             print("Warning: NaN/Inf detected in data_grid_tapered before FFT. Replacing with zeros.")
             data_grid_tapered = torch.nan_to_num(data_grid_tapered, nan=0.0, posinf=0.0, neginf=0.0)

        fft_results.append(torch.fft.fft2(data_grid_tapered))

    if not fft_results:
         print("Warning: No FFT results generated.")
         return torch.empty(0, 0, 0, device=device), n1, n2, 0, taper_grid

    J_vector_tensor = torch.stack(fft_results, dim=2).to(device)

    H = torch.sum(taper_grid**2)
    if H < 1e-12:
        print("Warning: Normalization factor H is near zero.")
        norm_factor = torch.tensor(0.0, device=device)
    else:
        norm_factor = (torch.sqrt(1.0 / H) / (2.0 * cmath.pi)).to(device)

    result = J_vector_tensor * norm_factor
    if torch.isnan(result).any(): print("Warning: NaN in J_vector output.")
    return result, n1, n2, p, taper_grid


def calculate_sample_periodogram_vectorized(J_vector_tensor):
    """Calculates sample periodogram I_n = J J^H (pxp matrix for each spatial freq)."""
    if torch.isnan(J_vector_tensor).any() or torch.isinf(J_vector_tensor).any():
        print("Warning: NaN/Inf detected in J_vector_tensor input.")
        n1, n2, p = J_vector_tensor.shape
        return torch.full((n1, n2, p, p), float('nan'), dtype=torch.complex64, device=J_vector_tensor.device)

    J_col = J_vector_tensor.unsqueeze(-1)
    J_row_conj = J_vector_tensor.unsqueeze(-2).conj()
    result = J_col @ J_row_conj

    if torch.isnan(result).any(): print("Warning: NaN in periodogram matrix output.")
    return result


# =========================================================================
# 4. Likelihood Calculation (Tapered)
# =========================================================================

def whittle_likelihood_loss_tapered(params, I_sample, n1, n2, p, taper_autocorr_grid, delta1, delta2):
    """
    ✅ Whittle Likelihood Loss using data tapering and exact taper autocorrelation c_gn.
    """
    device = I_sample.device
    params_tensor = params.to(device)

    if torch.isnan(params_tensor).any() or torch.isinf(params_tensor).any():
        print("Warning: NaN/Inf detected in input parameters to likelihood.")
        return torch.tensor(float('nan'), device=device)

    I_expected = expected_periodogram_fft_tapered(
        params_tensor, n1, n2, p, taper_autocorr_grid, 
        delta1, delta2
    )

    if torch.isnan(I_expected).any() or torch.isinf(I_expected).any():
        print("Warning: NaN/Inf returned from expected_periodogram calculation.")
        return torch.tensor(float('nan'), device=device)

    eye_matrix = torch.eye(p, dtype=torch.complex64, device=device)
    diag_vals = torch.abs(I_expected.diagonal(dim1=-2, dim2=-1))
    mean_diag_abs = diag_vals.mean().item() if diag_vals.numel() > 0 and not torch.isnan(diag_vals).all() else 1.0
    diag_load = max(mean_diag_abs * 1e-8, 1e-9)
    I_expected_stable = I_expected + eye_matrix * diag_load

    sign, logabsdet = torch.linalg.slogdet(I_expected_stable)
    if torch.any(sign.real <= 1e-9):
        print("Warning: Non-positive determinant encountered. Applying penalty.")
        log_det_term = torch.where(sign.real > 1e-9, logabsdet, torch.tensor(1e10, device=device))
    else:
        log_det_term = logabsdet

    if torch.isnan(I_sample).any() or torch.isinf(I_sample).any():
        print("Warning: NaN/Inf detected in I_sample input to likelihood.")
        return torch.tensor(float('nan'), device=device)

    try:
        solved_term = torch.linalg.solve(I_expected_stable, I_sample)
        trace_term = torch.einsum('...ii->...', solved_term).real
    except torch.linalg.LinAlgError as e:
        print(f"Warning: LinAlgError during solve: {e}. Applying high loss penalty.")
        return torch.tensor(float('inf'), device=device)

    if torch.isnan(trace_term).any() or torch.isinf(trace_term).any():
        print("Warning: NaN/Inf detected in trace_term. Returning NaN loss.")
        return torch.tensor(float('nan'), device=device)

    likelihood_terms = log_det_term + trace_term

    if torch.isnan(likelihood_terms).any():
        print("Warning: NaN detected in likelihood_terms before summation. Returning NaN loss.")
        return torch.tensor(float('nan'), device=device)

    total_sum = torch.sum(likelihood_terms)
    dc_term = likelihood_terms[0, 0] if n1 > 0 and n2 > 0 else torch.tensor(0.0, device=device)
    if torch.isnan(dc_term).any() or torch.isinf(dc_term).any():
        print("Warning: NaN/Inf detected in DC term. Setting to 0.")
        dc_term = torch.tensor(0.0, device=device)

    loss = total_sum - dc_term if (n1 > 1 or n2 > 1) else total_sum

    if torch.isnan(loss) or torch.isinf(loss):
         print("Warning: NaN/Inf detected in final loss. Returning Inf penalty.")
         return torch.tensor(float('inf'), device=device)

    return loss


# =========================================================================
# 5. Training Loop & Helpers (💥 REVISED 💥)
# =========================================================================

def get_printable_params(p_list):
    """Helper to convert log-params to natural scale for printing."""
    valid_tensors = [p for p in p_list if isinstance(p, torch.Tensor)]
    if not valid_tensors: return "Invalid params_list"
    p_cat = torch.cat([p.detach().clone().cpu() for p in valid_tensors])
    
    if len(p_cat) != 4:
        return f"Expected 4 params, got {len(p_cat)}."

    log_params = p_cat
    if torch.isnan(log_params).any() or torch.isinf(log_params).any():
        return "[NaN/Inf in log_params]"
        
    try:
        phi1 = torch.exp(log_params[0])
        phi2 = torch.exp(log_params[1]) # range_lon_inv
        phi3 = torch.exp(log_params[2]) # (range_lon / range_lat)^2
        nugget = torch.exp(log_params[3])

        epsilon = 1e-12
        sigmasq = phi1 / (phi2 + epsilon)
        range_lon = 1.0 / (phi2 + epsilon)
        range_lat = 1.0 / (torch.sqrt(phi3 + epsilon) * phi2 + epsilon)
        
        return (f"sigmasq: {sigmasq.item():.4f}, range_lat: {range_lat.item():.4f}, "
                f"range_lon: {range_lon.item():.4f}, nugget: {nugget.item():.4f}")
    except Exception as e:
        return f"[Error in param conversion: {e}]"

def get_phi_params(log_params_list):
    """Helper to print reparameterized (phi-scale) params."""
    try:
        p_cat = torch.cat([p.detach().clone().cpu() for p in log_params_list])
        phi1 = torch.exp(p_cat[0])
        phi2 = torch.exp(p_cat[1])
        phi3 = torch.exp(p_cat[2])
        nugget = torch.exp(p_cat[3]) 
        
        return (f"phi1: {phi1.item():.4f}, phi2: {phi2.item():.4f}, "
                f"phi3: {phi3.item():.4f}, nugget: {nugget.item():.4f}")
    except Exception:
        return "[Error in reparam conversion]"

# --- 💥 NEW HELPER 💥 ---
def get_raw_log_params(log_params_list):
    """Helper to print the raw log-phi params being optimized."""
    try:
        p_cat = torch.cat([p.detach().clone().cpu() for p in log_params_list])
        return (f"log_phi1: {p_cat[0].item():.4f}, log_phi2: {p_cat[1].item():.4f}, "
                f"log_phi3: {p_cat[2].item():.4f}, log_nugget: {p_cat[3].item():.4f}")
    except Exception:
        return "[Error in raw param conversion]"
# --- END NEW HELPER ---


def run_full_tapered(params_list, optimizer, scheduler, I_sample, n1, n2, p, taper_autocorr_grid, epochs=600, device='cpu'):
    """Corrected training loop with gradient-based convergence."""
    best_loss = float('inf')
    params_list = [p.to(device) for p in params_list]
    best_params_state = [p.detach().clone() for p in params_list]
    epochs_completed = 0
    DELTA_LAT, DELTA_LON = 0.044, 0.063 
    
    # --- 💥 NEW: Convergence tolerance 💥 ---
    grad_tol = 1e-5

    I_sample_dev = I_sample.to(device)
    taper_autocorr_grid_dev = taper_autocorr_grid.to(device) 

    for epoch in range(epochs):
        epochs_completed = epoch + 1
        optimizer.zero_grad()
        params_tensor = torch.cat(params_list) 

        loss = whittle_likelihood_loss_tapered(
            params_tensor, I_sample_dev, n1, n2, p, taper_autocorr_grid_dev, DELTA_LAT, DELTA_LON
        )

        if torch.isnan(loss) or torch.isinf(loss):
            print(f"Loss became NaN or Inf at epoch {epoch+1}. Stopping.")
            if epoch == 0: best_params_state = None
            epochs_completed = epoch
            break

        loss.backward()

        nan_grad = False
        max_abs_grad = 0.0
        for param in params_list:
            if param.grad is not None:
                if (torch.isnan(param.grad).any() or torch.isinf(param.grad).any()):
                    nan_grad = True
                    break
                # --- 💥 NEW: Track max gradient 💥 ---
                max_abs_grad = max(max_abs_grad, param.grad.abs().item())
            
        if nan_grad:
             print(f"Warning: NaN/Inf gradient at epoch {epoch+1}. Skipping step.")
             optimizer.zero_grad()
             continue

        all_params_on_device = params_list
        if all_params_on_device:
            torch.nn.utils.clip_grad_norm_(all_params_on_device, max_norm=1.0)
        
        # --- 💥 NEW: Convergence Check 💥 ---
        if epoch > 10 and max_abs_grad < grad_tol: # 10-epoch warmup
            print(f"\n--- Converged on gradient norm (max|grad| < {grad_tol}) at epoch {epoch+1} ---")
            epochs_completed = epoch + 1
            break # Exit the training loop
        # --- END NEW ---

        optimizer.step()
        scheduler.step()

        current_loss_item = loss.item()
        if current_loss_item < best_loss:
            params_valid = not any(torch.isnan(p.data).any() or torch.isinf(p.data).any() for p in params_list)
            if params_valid:
                best_loss = current_loss_item
                best_params_state = [p.detach().clone() for p in params_list]

        current_lr = optimizer.param_groups[0]['lr'] if optimizer.param_groups else 0.0

        if epoch % 50 == 0 or epoch == epochs - 1:
            print(f'--- Epoch {epoch+1}/{epochs} (LR: {current_lr:.6f}) ---')
            print(f' Loss: {current_loss_item:.4f} | Max Grad: {max_abs_grad:.6e}')
            print(f' Params (Raw Log Scale): {get_raw_log_params(params_list)}')
            print(f' Params (Natural Scale): {get_printable_params(params_list)}')


    print("\n--- Training Complete ---")
    if best_params_state is None:
        print("Training failed to find a valid model state.")
        return None, None, None, None, epochs_completed # Return Nones

    # Get final params in all 3 formats
    final_natural_params_str = get_printable_params(best_params_state)
    final_phi_params_str = get_phi_params(best_params_state)
    final_raw_params_str = get_raw_log_params(best_params_state)
    final_loss_rounded = round(best_loss, 3) if best_loss != float('inf') else float('inf')

    print(f'\nFINAL BEST STATE ACHIEVED (during training):')
    print(f'Best Loss: {final_loss_rounded}')
    print(f'Parameters (Natural Scale): {final_natural_params_str}')
    print(f'Parameters (Phi Scale)    : {final_phi_params_str}')
    print(f'Parameters (Raw Log Scale): {final_raw_params_str}')
    
    # Return strings for final printing
    return final_natural_params_str, final_phi_params_str, final_raw_params_str, final_loss_rounded, epochs_completed


# =========================================================================
# 6. Main Execution Script (💥 REVISED 💥)
# =========================================================================
if __name__ == '__main__':
    start_time = time.time()

    # --- Configuration ---
    DAY_TO_RUN = 1
    TAPERING_FUNC = cgn_hamming # Use Hamming taper
    NUM_RUNS = 1
    EPOCHS = 5000 
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {DEVICE}")

    DELTA_LAT, DELTA_LON = 0.044, 0.063 

    LAT_COL, LON_COL = 0, 1
    VAL_COL = 2 # Spatially differenced value
    TIME_COL = 3

    # --- Load Spatially Differenced Data ---
    try:
        with open("spatial_first_difference_data.pkl", 'rb') as f:
            processed_df = pickle.load(f)
        print(f"Loaded {len(processed_df)} days from spatial_first_difference_data.pkl.")
        processed_df = [
            torch.tensor(arr, dtype=torch.float32).cpu() if not isinstance(arr, torch.Tensor)
            else arr.cpu().to(torch.float32)
            for arr in processed_df
        ]
        if not processed_df: raise ValueError("'processed_df' is empty.")
    except FileNotFoundError:
        print("Error: `spatial_first_difference_data.pkl` not found.")
        exit()
    except Exception as e:
        print(f"Error loading or processing 'processed_df': {e}")
        exit()

    if DAY_TO_RUN > len(processed_df) or DAY_TO_RUN <= 0:
        print(f"Error: DAY_TO_RUN ({DAY_TO_RUN}) out of bounds.")
        exit()

    cur_df = processed_df[DAY_TO_RUN - 1]
    
    cur_df = cur_df[cur_df[:,3] == cur_df[:,3].min()]
    print(f"Filtered data to single time point. New shape: {cur_df.shape}")
    
    if cur_df.numel() == 0 or cur_df.shape[1] <= max(LAT_COL, LON_COL, VAL_COL, TIME_COL):
        print(f"Error: Data for Day {DAY_TO_RUN} (single time point) is empty or invalid.")
        exit()

    time_slices_list = [cur_df] # List containing the single time slice
    
    # --- 1. Pre-compute J-vector, Taper Grid, and Taper Autocorrelation ---
    print("Pre-computing J-vector (Hamming taper)...")
    J_vec, n1, n2, p, taper_grid = generate_Jvector_tapered( 
        time_slices_list,
        tapering_func=TAPERING_FUNC, 
        lat_col=LAT_COL, lon_col=LON_COL, val_col=VAL_COL,
        device=DEVICE
    )

    if J_vec is None or J_vec.numel() == 0 or n1 == 0 or n2 == 0 or p == 0:
       print(f"Error: J-vector generation failed for Day {DAY_TO_RUN}.")
       exit()
       
    print("Pre-computing sample periodogram...")
    I_sample = calculate_sample_periodogram_vectorized(J_vec)

    print("Pre-computing Hamming taper autocorrelation...")
    taper_autocorr_grid = calculate_taper_autocorrelation_fft(taper_grid, n1, n2, DEVICE)

    if torch.isnan(I_sample).any() or torch.isinf(I_sample).any():
        print("Error: NaN/Inf in sample periodogram.")
        exit()
    if torch.isnan(taper_autocorr_grid).any() or torch.isinf(taper_autocorr_grid).any():
        print("Error: NaN/Inf in taper autocorrelation.")
        exit()

    print(f"Data grid: {n1}x{n2}, {p} time points. J-vector, Periodogram, Taper Autocorr on {DEVICE}.")

    # --- 2. Optimization Loop ---
    all_final_results = []
    all_final_losses = []

    for i in range(NUM_RUNS):
        print(f"\n{'='*30} Initialization Run {i+1}/{NUM_RUNS} {'='*30}")

        # [log_phi1, log_phi2, log_phi3, log_nugget]
        initial_params_values = [np.log(15.0), np.log(0.66), np.log(0.7), np.log(1.5)]
        
        print(f"Starting with FIXED params (log-scale): {[round(p, 4) for p in initial_params_values]}")

        params_list = [
            Parameter(torch.tensor([val], dtype=torch.float32))
            for val in initial_params_values
        ]

        lr = 0.01 
        optimizer = torch.optim.Adam(params_list, lr=lr)

        T_MAX = 200
        ETA_MIN = 1e-6
        scheduler = CosineAnnealingLR(optimizer, T_max=T_MAX, eta_min=ETA_MIN)

        print(f"Starting optimization run {i+1} on device {DEVICE} (Hamming, SpatialModel kernel)...")
        
        # --- 💥 REVISED: Unpack 5 return values 💥 ---
        nat_params_str, phi_params_str, raw_params_str, loss, epochs_run = run_full_tapered(
            params_list=params_list,
            optimizer=optimizer,
            scheduler=scheduler,
            I_sample=I_sample,
            n1=n1, n2=n2, p=p,
            taper_autocorr_grid=taper_autocorr_grid, 
            epochs=EPOCHS,
            device=DEVICE
        )
        
        if loss is not None:
            all_final_results.append((nat_params_str, phi_params_str, raw_params_str))
            all_final_losses.append(loss)
        else:
            all_final_losses.append(float('inf'))

    print(f"\n\g{'='*25} Overall Result from Run {'='*25}")
    valid_losses = [l for l in all_final_losses if l is not None and l != float('inf')]

    if not valid_losses:
        print(f"The run failed or resulted in an invalid loss for Day {DAY_TO_RUN}.")
    else:
        best_loss = min(valid_losses)
        best_run_index = all_final_losses.index(best_loss)
        best_results = all_final_results[best_run_index]
        
        # --- 💥 REVISED: Print all 3 param scales 💥 ---
        print(f"Best Run Loss: {best_loss}")
        print(f"Final Parameters (Natural Scale): {best_results[0]}")
        print(f"Final Parameters (Phi Scale)    : {best_results[1]}")
        print(f"Final Parameters (Raw Log Scale): {best_results[2]}")

    end_time = time.time()
    print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")

<>:676: SyntaxWarning: invalid escape sequence '\g'
<>:676: SyntaxWarning: invalid escape sequence '\g'
/var/folders/9p/53hd4c7d2fl193h4jwp194wc0000gn/T/ipykernel_41423/2319905123.py:676: SyntaxWarning: invalid escape sequence '\g'
  print(f"\n\g{'='*25} Overall Result from Run {'='*25}")


Using device: cpu
Loaded 31 days from spatial_first_difference_data.pkl.
Filtered data to single time point. New shape: torch.Size([17854, 4])
Pre-computing J-vector (Hamming taper)...
Pre-computing sample periodogram...
Pre-computing Hamming taper autocorrelation...
Data grid: 113x158, 1 time points. J-vector, Periodogram, Taper Autocorr on cpu.

============================== Initialization Run 1/1 ==============================
Starting with FIXED params (log-scale): [2.7081, -0.4155, -0.3567, 0.4055]
Starting optimization run 1 on device cpu (Hamming, SpatialModel kernel)...
--- Epoch 1/5000 (LR: 0.009999) ---
 Loss: 8893.6602 | Max Grad: 8.774561e+03
 Params (Raw Log Scale): log_phi1: 2.7181, log_phi2: -0.4255, log_phi3: -0.3467, log_nugget: 0.4155
 Params (Natural Scale): sigmasq: 23.1864, range_lat: 1.8200, range_lon: 1.5304, nugget: 1.5151
--- Epoch 51/5000 (LR: 0.008480) ---
 Loss: 4808.0454 | Max Grad: 6.999688e+02
 Params (Raw Log Scale): log_phi1: 3.2090, log_phi2: -0.7884,

# Compute NLL

In [5]:
import torch
import numpy as np
import matplotlib.pyplot as plt 
import cmath
import pickle
import time 
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.nn import Parameter
import torch.fft 
from typing import List, Dict, Any, Callable

# =========================================================================
# ALL HELPER FUNCTIONS FROM YOUR REFERENCE CODE
# (Copied verbatim)
# =========================================================================

def cgn_hamming(u, n1, n2):
    """Computes a 2D Hamming window."""
    u1, u2 = u
    device = u1.device if isinstance(u1, torch.Tensor) else (u2.device if isinstance(u2, torch.Tensor) else torch.device('cpu'))
    u1_tensor = u1.to(device) if isinstance(u1, torch.Tensor) else torch.tensor(u1, device=device, dtype=torch.float32)
    u2_tensor = u2.to(device) if isinstance(u2, torch.Tensor) else torch.tensor(u2, device=device, dtype=torch.float32)

    n1_eff = float(n1) if n1 > 0 else 1.0
    n2_eff = float(n2) if n2 > 0 else 1.0
    hamming1 = 0.54 + 0.46 * torch.cos(2.0 * torch.pi * u1_tensor / n1_eff)
    hamming2 = 0.54 + 0.46 * torch.cos(2.0 * torch.pi * u2_tensor / n2_eff)
    return hamming1 * hamming2

def cgn_2dbartlett(u, n1, n2): 
    """Computes a 2D Bartlett window function."""
    u1, u2 = u
    device = u1.device if isinstance(u1, torch.Tensor) else (u2.device if isinstance(u2, torch.Tensor) else torch.device('cpu'))
    u1_tensor = u1.to(device) if isinstance(u1, torch.Tensor) else torch.tensor(u1, device=device, dtype=torch.float32)
    u2_tensor = u2.to(device) if isinstance(u2, torch.Tensor) else torch.tensor(u2, device=device, dtype=torch.float32)
    n1_eff = float(n1) if n1 > 0 else 1.0
    n2_eff = float(n2) if n2 > 0 else 1.0
    kernel = (1.0 - torch.abs(u1_tensor) / n1_eff) * (1.0 - torch.abs(u2_tensor) / n2_eff)
    return torch.clamp(kernel, min=0.0)

def calculate_taper_autocorrelation_fft(taper_grid, n1, n2, device):
    """
    Computes the normalized taper autocorrelation function c_gn(u) using FFT.
    """
    taper_grid = taper_grid.to(device) 
    H = torch.sum(taper_grid**2)
    if H < 1e-12:
        print("Warning: Sum of squared taper weights (H) is near zero.")
        return torch.zeros((2*n1-1, 2*n2-1), device=device, dtype=taper_grid.dtype)

    N1, N2 = 2 * n1 - 1, 2 * n2 - 1
    taper_fft = torch.fft.fft2(taper_grid, s=(N1, N2))
    power_spectrum = torch.abs(taper_fft)**2
    autocorr_unnormalized = torch.fft.ifft2(power_spectrum).real
    autocorr_shifted = torch.fft.fftshift(autocorr_unnormalized)

    c_gn_grid = autocorr_shifted / (H + 1e-12)

    return c_gn_grid 

def cov_x_spatial_model_kernel(u1, u2, t, params):
    """
    Computes autocovariance of X using the SPATIAL-ONLY kernel.
    u1, u2 are PHYSICAL lags (already scaled by deltas).
    """
    device = params.device
    u1_dev = u1.to(device) if isinstance(u1, torch.Tensor) else torch.tensor(u1, device=device, dtype=torch.float32)
    u2_dev = u2.to(device) if isinstance(u2, torch.Tensor) else torch.tensor(u2, device=device, dtype=torch.float32)
    t_dev = t.to(device) if isinstance(t, torch.Tensor) else torch.tensor(t, device=device, dtype=torch.float32)

    if torch.isnan(params).any() or torch.isinf(params).any():
         out_shape = torch.broadcast_shapes(u1_dev.shape, u2_dev.shape, t_dev.shape)
         return torch.full(out_shape, float('nan'), device=device, dtype=torch.float32)

    phi1   = torch.exp(params[0])
    phi2   = torch.exp(params[1]) 
    phi3   = torch.exp(params[2]) 
    nugget = torch.exp(params[3])

    epsilon = 1e-12
    sigmasq = phi1 / (phi2 + epsilon)  
    range_lon_inv = phi2
    range_lat_inv = torch.sqrt(phi3 + epsilon) * phi2

    dist_sq = (u1_dev * range_lat_inv).pow(2) + (u2_dev * range_lon_inv).pow(2)
    distance = torch.sqrt(dist_sq + epsilon) 

    cov_smooth = sigmasq * torch.exp(-distance)

    is_zero_lag = (torch.abs(u1_dev) < 1e-9) & (torch.abs(u2_dev) < 1e-9) & (torch.abs(t_dev) < 1e-9)
    final_cov = torch.where(is_zero_lag, cov_smooth + nugget, cov_smooth)

    if torch.isnan(final_cov).any(): print("Warning: NaN detected in cov_x_spatial_model_kernel output.")
    return final_cov


def cov_spatial_difference(u1, u2, t, params, delta1, delta2):
    """
    Calculates covariance Cov(Y(s), Y(s+u)) for Y(s) = X(s+d1) + X(s+d2) - 2X(s)
    Based on the underlying spatial kernel.
    u1, u2 are PHYSICAL lags.
    """
    weights = {(0, 0): -2.0, (1, 0): 1.0, (0, 1): 1.0}
    device = params.device
    out_shape = torch.broadcast_shapes(u1.shape if isinstance(u1, torch.Tensor) else (),
                                     u2.shape if isinstance(u2, torch.Tensor) else (),
                                     t.shape if isinstance(t, torch.Tensor) else ())
    cov = torch.zeros(out_shape, device=device, dtype=torch.float32)
    u1_dev = u1.to(device) if isinstance(u1, torch.Tensor) else torch.tensor(u1, device=device, dtype=torch.float32)
    u2_dev = u2.to(device) if isinstance(u2, torch.Tensor) else torch.tensor(u2, device=device, dtype=torch.float32)
    t_dev = t.to(device) if isinstance(t, torch.Tensor) else torch.tensor(t, device=device, dtype=torch.float32)

    for (a_idx, b_idx), w_ab in weights.items():
        offset_a1 = a_idx * delta1
        offset_a2 = b_idx * delta2
        for (c_idx, d_idx), w_cd in weights.items():
            offset_c1 = c_idx * delta1
            offset_c2 = d_idx * delta2
            
            lag_u1 = u1_dev + (offset_a1 - offset_c1)
            lag_u2 = u2_dev + (offset_a2 - offset_c2)
            
            term_cov = cov_x_spatial_model_kernel(lag_u1, lag_u2, t_dev, params) 
            
            if torch.isnan(term_cov).any():
                 print(f"Warning: NaN in term_cov within cov_spatial_difference.")
                 return torch.full_like(cov, float('nan'))
            cov += w_ab * w_cd * term_cov

    if torch.isnan(cov).any(): print("Warning: NaN in final cov_spatial_difference output.")
    return cov

def cn_bar_tapered(u1, u2, t, params, n1, n2, taper_autocorr_grid, delta1, delta2):
    """
    Computes c_Y(u) * c_gn(u).
    u1, u2 are GRID index lags (e.g., -n1..0..n1)
    """
    device = params.device
    u1_dev = u1.to(device) if isinstance(u1, torch.Tensor) else torch.tensor(u1, device=device, dtype=torch.float32)
    u2_dev = u2.to(device) if isinstance(u2, torch.Tensor) else torch.tensor(u2, device=device, dtype=torch.float32)
    t_dev = t.to(device) if isinstance(t, torch.Tensor) else torch.tensor(t, device=device, dtype=torch.float32)

    # --- Convert GRID lags to PHYSICAL lags ---
    lag_u1 = u1_dev * delta1
    lag_u2 = u2_dev * delta2
    
    cov_X_value = cov_spatial_difference(lag_u1, lag_u2, t_dev, params, delta1, delta2)

    # --- Get Taper Autocorrelation Value c_gn(u) from grid ---
    u1_idx = u1_dev.long()
    u2_idx = u2_dev.long()

    idx1 = (n1 - 1 + u1_idx) # Centering index
    idx2 = (n2 - 1 + u2_idx) # Centering index
    
    idx1 = torch.clamp(idx1, 0, 2 * n1 - 2)
    idx2 = torch.clamp(idx2, 0, 2 * n2 - 2)

    taper_autocorr_value = taper_autocorr_grid[idx1, idx2]

    if torch.isnan(cov_X_value).any() or torch.isnan(taper_autocorr_value).any():
        out_shape = torch.broadcast_shapes(cov_X_value.shape, taper_autocorr_value.shape)
        return torch.full(out_shape, float('nan'), device=device, dtype=torch.float32)

    result = cov_X_value * taper_autocorr_value
    if torch.isnan(result).any(): print("Warning: NaN in cn_bar_tapered output.")
    return result


def expected_periodogram_fft_tapered(params, n1, n2, p, taper_autocorr_grid, delta1, delta2):
    """
    Calculates the expected periodogram I(omega_s) (a pxp matrix in time)
    using the exact taper autocorrelation c_gn(u) and
    CORRECTLY implementing the aliasing sum (Lemma 2).
    """
    device = params.device if isinstance(params, torch.Tensor) else params[0].device
    if isinstance(params, list):
        params_tensor = torch.cat([p.to(device) for p in params])
    else:
        params_tensor = params.to(device)

    # This function uses positive grid index lags [0, n-1] for the sum
    u1_lags = torch.arange(n1, dtype=torch.float32, device=device)
    u2_lags = torch.arange(n2, dtype=torch.float32, device=device)
    u1_mesh, u2_mesh = torch.meshgrid(u1_lags, u2_lags, indexing='ij')

    t_lags = torch.arange(p, dtype=torch.float32, device=device)
    tilde_cn_tensor = torch.zeros((n1, n2, p, p), dtype=torch.complex64, device=device)

    for q in range(p):
        for r in range(p):
            t_diff = t_lags[q] - t_lags[r]
            
            # These are the four terms for the aliasing sum
            term1 = cn_bar_tapered(u1_mesh, u2_mesh, t_diff, 
                                   params_tensor, n1, n2, taper_autocorr_grid, delta1, delta2)
            term2 = cn_bar_tapered(u1_mesh - n1, u2_mesh, t_diff, 
                                   params_tensor, n1, n2, taper_autocorr_grid, delta1, delta2)
            term3 = cn_bar_tapered(u1_mesh, u2_mesh - n2, t_diff, 
                                   params_tensor, n1, n2, taper_autocorr_grid, delta1, delta2)
            term4 = cn_bar_tapered(u1_mesh - n1, u2_mesh - n2, t_diff,
                                   params_tensor, n1, n2, taper_autocorr_grid, delta1, delta2)
            
            tilde_cn_grid_qr = (term1 + term2 + term3 + term4)
            
            if torch.isnan(tilde_cn_grid_qr).any():
                 tilde_cn_tensor[:, :, q, r] = float('nan')
            else:
                 tilde_cn_tensor[:, :, q, r] = tilde_cn_grid_qr.to(torch.complex64)

    if torch.isnan(tilde_cn_tensor).any():
        print("Warning: NaN detected in tilde_cn_tensor before FFT.")
        nan_shape = (n1, n2, p, p)
        return torch.full(nan_shape, float('nan'), dtype=torch.complex64, device=device)

    fft_result = torch.fft.fft2(tilde_cn_tensor, dim=(0, 1))
    fft_result_real = fft_result.real 
    normalization_factor = 1.0 / (4.0 * cmath.pi**2)
    result = fft_result_real * normalization_factor

    if torch.isnan(result).any(): print("Warning: NaN in expected_periodogram_fft_tapered output.")
    return result

def generate_Jvector_tapered(tensor_list, tapering_func, lat_col, lon_col, val_col, device):
    """
    Generates J-vector for a single component using the specified taper,
    placing result on device.
    """
    p = len(tensor_list)
    if p == 0: return torch.empty(0, 0, 0, device=device), 0, 0, 0, None 

    valid_tensors = [t for t in tensor_list if t.numel() > 0 and t.shape[1] > max(lat_col, lon_col, val_col)]
    if not valid_tensors:
         print("Warning: No valid tensors found in tensor_list.")
         return torch.empty(0, 0, 0, device=device), 0, 0, 0, None

    try:
        all_lats_cpu = torch.cat([t[:, lat_col] for t in valid_tensors])
        all_lons_cpu = torch.cat([t[:, lon_col] for t in valid_tensors])
    except IndexError:
        print(f"Error: Invalid column index. Check tensor shapes.")
        return torch.empty(0, 0, 0, device=device), 0, 0, 0, None

    all_lats_cpu = all_lats_cpu[~torch.isnan(all_lats_cpu) & ~torch.isinf(all_lats_cpu)]
    all_lons_cpu = all_lons_cpu[~torch.isnan(all_lons_cpu) & ~torch.isinf(all_lons_cpu)]
    if all_lats_cpu.numel() == 0 or all_lons_cpu.numel() == 0:
        print("Warning: No valid coordinates after NaN/Inf filtering.")
        return torch.empty(0, 0, 0, device=device), 0, 0, 0, None

    unique_lats_cpu, unique_lons_cpu = torch.unique(all_lats_cpu), torch.unique(all_lons_cpu)
    n1, n2 = len(unique_lats_cpu), len(unique_lons_cpu)
    if n1 == 0 or n2 == 0:
        print("Warning: Grid dimensions are zero.")
        return torch.empty(0, 0, 0, device=device), 0, 0, 0, None

    lat_map = {lat.item(): i for i, lat in enumerate(unique_lats_cpu)}
    lon_map = {lon.item(): i for i, lon in enumerate(unique_lons_cpu)}

    u1_mesh_cpu, u2_mesh_cpu = torch.meshgrid(
        torch.arange(n1, dtype=torch.float32),
        torch.arange(n2, dtype=torch.float32),
        indexing='ij'
    )
    taper_grid = tapering_func((u1_mesh_cpu, u2_mesh_cpu), n1, n2).to(device) # Taper on device

    fft_results = []
    for tensor in tensor_list:
        data_grid = torch.zeros((n1, n2), dtype=torch.float32, device=device)
        if tensor.numel() > 0 and tensor.shape[1] > max(lat_col, lon_col, val_col):
            for row in tensor:
                lat_item, lon_item = row[lat_col].item(), row[lon_col].item()
                if not (np.isnan(lat_item) or np.isnan(lon_item)):
                    i = lat_map.get(lat_item)
                    j = lon_map.get(lon_item)
                    if i is not None and j is not None:
                        val = row[val_col]
                        val_num = val.item() if isinstance(val, torch.Tensor) else val
                        if not np.isnan(val_num) and not np.isinf(val_num):
                           data_grid[i, j] = val_num

        data_grid_tapered = data_grid * taper_grid 

        if torch.isnan(data_grid_tapered).any() or torch.isinf(data_grid_tapered).any():
             print("Warning: NaN/Inf detected in data_grid_tapered before FFT. Replacing with zeros.")
             data_grid_tapered = torch.nan_to_num(data_grid_tapered, nan=0.0, posinf=0.0, neginf=0.0)

        fft_results.append(torch.fft.fft2(data_grid_tapered))

    if not fft_results:
         print("Warning: No FFT results generated.")
         return torch.empty(0, 0, 0, device=device), n1, n2, 0, taper_grid

    J_vector_tensor = torch.stack(fft_results, dim=2).to(device)

    H = torch.sum(taper_grid**2)
    if H < 1e-12:
        print("Warning: Normalization factor H is near zero.")
        norm_factor = torch.tensor(0.0, device=device)
    else:
        norm_factor = (torch.sqrt(1.0 / H) / (2.0 * cmath.pi)).to(device)

    result = J_vector_tensor * norm_factor
    if torch.isnan(result).any(): print("Warning: NaN in J_vector output.")
    return result, n1, n2, p, taper_grid


def calculate_sample_periodogram_vectorized(J_vector_tensor):
    """Calculates sample periodogram I_n = J J^H (pxp matrix for each spatial freq)."""
    if torch.isnan(J_vector_tensor).any() or torch.isinf(J_vector_tensor).any():
        print("Warning: NaN/Inf detected in J_vector_tensor input.")
        n1, n2, p = J_vector_tensor.shape
        return torch.full((n1, n2, p, p), float('nan'), dtype=torch.complex64, device=J_vector_tensor.device)

    J_col = J_vector_tensor.unsqueeze(-1)
    J_row_conj = J_vector_tensor.unsqueeze(-2).conj()
    result = J_col @ J_row_conj

    if torch.isnan(result).any(): print("Warning: NaN in periodogram matrix output.")
    return result

def whittle_likelihood_loss_tapered(params, I_sample, n1, n2, p, taper_autocorr_grid, delta1, delta2):
    """
    ✅ Whittle Likelihood Loss using data tapering and exact taper autocorrelation c_gn.
    """
    device = I_sample.device
    params_tensor = params.to(device)

    if torch.isnan(params_tensor).any() or torch.isinf(params_tensor).any():
        print("Warning: NaN/Inf detected in input parameters to likelihood.")
        return torch.tensor(float('nan'), device=device)

    I_expected = expected_periodogram_fft_tapered(
        params_tensor, n1, n2, p, taper_autocorr_grid, 
        delta1, delta2
    )

    if torch.isnan(I_expected).any() or torch.isinf(I_expected).any():
        print("Warning: NaN/Inf returned from expected_periodogram calculation.")
        return torch.tensor(float('nan'), device=device)

    eye_matrix = torch.eye(p, dtype=torch.complex64, device=device)
    diag_vals = torch.abs(I_expected.diagonal(dim1=-2, dim2=-1))
    mean_diag_abs = diag_vals.mean().item() if diag_vals.numel() > 0 and not torch.isnan(diag_vals).all() else 1.0
    diag_load = max(mean_diag_abs * 1e-8, 1e-9)
    I_expected_stable = I_expected + eye_matrix * diag_load

    sign, logabsdet = torch.linalg.slogdet(I_expected_stable)
    if torch.any(sign.real <= 1e-9):
        print("Warning: Non-positive determinant encountered. Applying penalty.")
        log_det_term = torch.where(sign.real > 1e-9, logabsdet, torch.tensor(1e10, device=device))
    else:
        log_det_term = logabsdet

    if torch.isnan(I_sample).any() or torch.isinf(I_sample).any():
        print("Warning: NaN/Inf detected in I_sample input to likelihood.")
        return torch.tensor(float('nan'), device=device)

    try:
        solved_term = torch.linalg.solve(I_expected_stable, I_sample)
        trace_term = torch.einsum('...ii->...', solved_term).real
    except torch.linalg.LinAlgError as e:
        print(f"Warning: LinAlgError during solve: {e}. Applying high loss penalty.")
        return torch.tensor(float('inf'), device=device)

    if torch.isnan(trace_term).any() or torch.isinf(trace_term).any():
        print("Warning: NaN/Inf detected in trace_term. Returning NaN loss.")
        return torch.tensor(float('nan'), device=device)

    likelihood_terms = log_det_term + trace_term

    if torch.isnan(likelihood_terms).any():
        print("Warning: NaN detected in likelihood_terms before summation. Returning NaN loss.")
        return torch.tensor(float('nan'), device=device)

    total_sum = torch.sum(likelihood_terms)
    dc_term = likelihood_terms[0, 0] if n1 > 0 and n2 > 0 else torch.tensor(0.0, device=device)
    if torch.isnan(dc_term).any() or torch.isinf(dc_term).any():
        print("Warning: NaN/Inf detected in DC term. Setting to 0.")
        dc_term = torch.tensor(0.0, device=device)

    loss = total_sum - dc_term if (n1 > 1 or n2 > 1) else total_sum

    if torch.isnan(loss) or torch.isinf(loss):
         print("Warning: NaN/Inf detected in final loss. Returning Inf penalty.")
         return torch.tensor(float('inf'), device=device)

    return loss

# =========================================================================
# 6. NEW NLL CALCULATOR CLASS
# =========================================================================

class WhittleNLLCalculator:
    """
    A class to pre-load data and compute the Whittle NLL for given parameters.
    
    This class assumes the 4-parameter SPATIAL model from the reference code.
    """
    def __init__(self, pickle_file_path, day_to_run, device='cpu'):
        print("Initializing WhittleNLLCalculator...")
        self.device = torch.device(device)
        self.DELTA_LAT, self.DELTA_LON = 0.044, 0.063
        self.tapering_func = cgn_hamming
        self.LAT_COL, self.LON_COL = 0, 1
        self.VAL_COL = 2 
        self.TIME_COL = 3
        
        # --- 1. Load and Process Data ---
        print(f"Loading data for Day {day_to_run} from {pickle_file_path}...")
        try:
            with open(pickle_file_path, 'rb') as f:
                processed_df = pickle.load(f)
            processed_df = [
                torch.tensor(arr, dtype=torch.float32).cpu() if not isinstance(arr, torch.Tensor)
                else arr.cpu().to(torch.float32)
                for arr in processed_df
            ]
        except Exception as e:
            print(f"Error loading pickle file: {e}")
            raise
            
        cur_df = processed_df[day_to_run - 1]
        
        # Filter to a single time point (p=1) as in the reference
        cur_df = cur_df[cur_df[:,3] == cur_df[:,3].min()]
        print(f"Filtered data to single time point. Shape: {cur_df.shape}")
        
        if cur_df.numel() == 0:
            print("Error: No data found for this day and time point.")
            raise ValueError("No data found")
            
        time_slices_list = [cur_df]
        
        # --- 2. Pre-compute J-vector and Taper Autocorrelation ---
        print("Pre-computing J-vector, Periodogram, and Taper Autocorrelation...")
        J_vec, n1, n2, p, taper_grid = generate_Jvector_tapered( 
            time_slices_list,
            tapering_func=self.tapering_func, 
            lat_col=self.LAT_COL, lon_col=self.LON_COL, 
            val_col=self.VAL_COL,
            device=self.device
        )
        
        if J_vec is None or J_vec.numel() == 0:
           print(f"Error: J-vector generation failed.")
           raise RuntimeError("J-vector generation failed")

        self.n1, self.n2, self.p = n1, n2, p
        
        # Store the pre-computed data-dependent objects
        self.I_sample = calculate_sample_periodogram_vectorized(J_vec)
        self.taper_autocorr_grid = calculate_taper_autocorrelation_fft(taper_grid, n1, n2, self.device)

        if torch.isnan(self.I_sample).any():
            raise ValueError("NaN detected in sample periodogram (I_sample).")
        if torch.isnan(self.taper_autocorr_grid).any():
            raise ValueError("NaN detected in taper autocorrelation grid.")

        print(f"Initialization complete. Grid: {n1}x{n2}, Time: {p}.")

    def calculate_nll(self, log_phi1: float, log_phi2: float, log_phi3: float, log_nugget: float) -> float:
        """
        Calculates the debiased Whittle NLL for a given set of parameters.
        
        Args:
            log_phi1 (float): Raw log(sigmasq / range_lon)
            log_phi2 (float): Raw log(1 / range_lon)
            log_phi3 (float): Raw log((range_lon / range_lat)^2)
            log_nugget (float): Raw log(nugget)
            
        Returns:
            float: The computed Negative Log-Likelihood (NLL).
        """
        
        # 1. Convert Python floats to a single Torch tensor
        params_tensor = torch.tensor(
            [log_phi1, log_phi2, log_phi3, log_nugget], 
            dtype=torch.float32, 
            device=self.device
        )
        
        # 2. Call the NLL loss function from your reference code
        loss = whittle_likelihood_loss_tapered(
            params=params_tensor, 
            I_sample=self.I_sample, 
            n1=self.n1, 
            n2=self.n2, 
            p=self.p, 
            taper_autocorr_grid=self.taper_autocorr_grid, 
            delta1=self.DELTA_LAT, 
            delta2=self.DELTA_LON
        )
        
        # 3. Return the NLL as a simple Python float
        return loss.item()



In [12]:
# =========================================================================
# 7. EXAMPLE USAGE
# =========================================================================
if __name__ == '__main__':
    
    # --- This is an example of how to use the new function ---
    
    # 1. SETTINGS
    PICKLE_PATH = "spatial_first_difference_data.pkl"
    DAY = 1
    
    # Set the 4 parameters you want to test
    # (Using the initial values from your reference code as an example)
    log_phi1_test = np.log(3.7371)
    log_phi2_test = np.log(1.3191)
    log_phi3_test = np.log(0.8742)
    log_nugget_test = np.log(0.4129)

    try:
        # 2. Initialize the calculator ONCE. 
        # This loads data and does all pre-computation.
        calculator = WhittleNLLCalculator(
            pickle_file_path=PICKLE_PATH, 
            day_to_run=DAY,
            device='cpu' # Use 'cuda' if available
        )
        
        # 3. Call the .calculate_nll() method with your parameters.
        # This part is fast and can be called many times.
        print("\n--- Calculating NLL at test parameters ---")
        nll_value = calculator.calculate_nll(
            log_phi1=log_phi1_test,
            log_phi2=log_phi2_test,
            log_phi3=log_phi3_test,
            log_nugget=log_nugget_test
        )
        
        print(f"\nFINAL NLL VALUE: {nll_value}")

        # You can now test another point quickly:
        print("\n--- Calculating NLL at another point ---")
        nll_value_2 = calculator.calculate_nll(
            log_phi1=np.log(20.0), # Changed sigmasq
            log_phi2=log_phi2_test,
            log_phi3=log_phi3_test,
            log_nugget=log_nugget_test
        )
        print(f"FINAL NLL VALUE (2): {nll_value_2}")

    except FileNotFoundError:
        print(f"\nERROR: Could not find the data file '{PICKLE_PATH}'.")
        print("Please make sure the file is in the same directory.")
    except Exception as e:
        print(f"\nAn error occurred: {e}")

Initializing WhittleNLLCalculator...
Loading data for Day 1 from spatial_first_difference_data.pkl...
Filtered data to single time point. Shape: torch.Size([17854, 4])
Pre-computing J-vector, Periodogram, and Taper Autocorrelation...
Initialization complete. Grid: 113x158, Time: 1.

--- Calculating NLL at test parameters ---

FINAL NLL VALUE: 71326.2890625

--- Calculating NLL at another point ---
FINAL NLL VALUE (2): 19477.99609375


# simulation study

hamming tapering

In [32]:
import torch
import numpy as np
import matplotlib.pyplot as plt 
import cmath
import pickle
import time 
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.nn import Parameter
import torch.fft 
from typing import List, Dict, Any, Callable

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# =========================================================================
# 1. Data Generation and Model Functions
# =========================================================================

def convert_to_log_reparam(sigmasq, range_lat, range_lon, nugget):
    """
    Converts physical parameters to the 4-parameter log-reparameterization.
    """
    device = sigmasq.device
    epsilon = 1e-12
    
    range_lon_inv = 1.0 / (range_lon + epsilon)
    log_phi2 = torch.log(range_lon_inv)
    log_phi1 = torch.log(sigmasq * range_lon_inv + epsilon)
    log_phi3 = torch.log((range_lon / range_lat).pow(2) + epsilon)
    log_nugget = torch.log(nugget + epsilon)
    
    log_params = torch.tensor([log_phi1, log_phi2, log_phi3, log_nugget], device=device)
    return log_params


def build_cov_matrix(locs, log_params):
    """
    Builds the full N x N covariance matrix from the SpatialModel's
    4-parameter reparameterization.
    
    locs: (N, 2) tensor of [lat, lon] coordinates
    log_params: (4,) tensor of [log_phi1, log_phi2, log_phi3, log_nugget]
    """
    device = locs.device
    
    # --- A. Recover all parameters ---
    phi1   = torch.exp(log_params[0])
    phi2   = torch.exp(log_params[1]) # This is range_lon_inv
    phi3   = torch.exp(log_params[2]) # This is (range_lon / range_lat)^2
    nugget = torch.exp(log_params[3])

    # --- B. Derive Physical Parameters ---
    epsilon = 1e-12
    # Ensure phi2 is not zero to avoid division by zero
    phi2_safe = phi2 + epsilon 
    sigmasq = phi1 / phi2_safe
    range_lon_inv = phi2
    range_lat_inv = torch.sqrt(phi3 + epsilon) * phi2

    # --- C. Calculate Anisotropic Spatial Distance ---
    x_lat, x_lon = locs[:, 0], locs[:, 1]
    
    delta_lat = x_lat.unsqueeze(1) - x_lat.unsqueeze(0)
    delta_lon = x_lon.unsqueeze(1) - x_lon.unsqueeze(0)

    dist_sq = (delta_lat * range_lat_inv).pow(2) + (delta_lon * range_lon_inv).pow(2)
    distance = torch.sqrt(dist_sq + epsilon) 

    # --- D. Calculate Covariance (Matern 0.5 = Exponential) ---
    cov = sigmasq * torch.exp(-distance)

    # --- E. Add Nugget ---
    cov.diagonal().add_(nugget + epsilon) # Add jitter to nugget
    
    return cov


def generate_spatial_data(n1, n2, lat_range, lon_range, true_params, device):
    """
    Generates a single n1 x n2 spatial field.
    """
    print(f"Generating spatial data ({n1} lats x {n2} lons)...")
    N = n1 * n2
    
    lats = torch.linspace(lat_range[0], lat_range[1], n1, device=device, dtype=torch.float64)
    lons = torch.linspace(lon_range[0], lon_range[1], n2, device=device, dtype=torch.float64)
    grid_lats, grid_lons = torch.meshgrid(lats, lons, indexing='ij')
    
    locs = torch.stack([grid_lats.ravel(), grid_lons.ravel()], dim=1)
    
    log_params_true = convert_to_log_reparam(
        torch.tensor(true_params['sigmasq'], device=device, dtype=torch.float64), 
        torch.tensor(true_params['range_lat'], device=device, dtype=torch.float64),
        torch.tensor(true_params['range_lon'], device=device, dtype=torch.float64),
        torch.tensor(true_params['nugget'], device=device, dtype=torch.float64)
    )
    
    print(f"Building N x N covariance matrix (N={N})...")
    K = build_cov_matrix(locs, log_params_true)
    
    print("Performing Cholesky decomposition...")
    try:
        L = torch.linalg.cholesky(K)
    except torch.linalg.LinAlgError:
        print("Warning: Cholesky failed. Adding jitter.")
        L = torch.linalg.cholesky(K + torch.eye(N, device=device, dtype=torch.float64) * 1e-5)
        
    z = torch.randn(N, 1, device=device, dtype=torch.float64)
    # Return a flat (N, 1) vector and the (N, 2) locations
    data_vec = (L @ z) 
    
    print("Data generated.")
    return data_vec.float(), locs.float() # Return N,1 data and N,2 locs


# =========================================================================
# 2. Full Maximum Likelihood Loss (No Tapering)
# =========================================================================

def full_nll_loss(log_params, data_vec, locs):
    """
    Computes the exact Negative Log-Likelihood (NLL) for a Gaussian Process.
    
    log_params: (4,) tensor of [log_phi1, log_phi2, log_phi3, log_nugget]
    data_vec: (N, 1) tensor of observations
    locs: (N, 2) tensor of [lat, lon] coordinates
    """
    N = data_vec.shape[0]
    device = data_vec.device
    
    # 1. Build the covariance matrix from the current parameters
    # We must rebuild this every time for gradient calculation
    try:
        K = build_cov_matrix(locs, log_params)
    except Exception as e:
        print(f"Warning: Error in build_cov_matrix: {e}")
        return torch.tensor(float('inf'), device=device)

    # 2. Compute the Cholesky decomposition
    try:
        # Add small jitter for numerical stability during optimization
        jitter = torch.eye(N, device=device, dtype=K.dtype) * 1e-6
        L = torch.linalg.cholesky(K + jitter)
    except torch.linalg.LinAlgError:
        print("Warning: Cholesky decomposition failed.")
        # Return a very large loss to penalize these parameters
        return torch.tensor(float('inf'), device=device)

    # 3. Calculate the log-determinant term
    # log(det(K)) = 2 * sum(log(diag(L)))
    log_det_K = 2 * torch.sum(torch.log(torch.diag(L)))

    # 4. Calculate the quadratic form term (y^T * K^{-1} * y)
    # Solve L*v = y for v
    v = torch.cholesky_solve(data_vec, L, upper=False)
    # Quad form is y^T * v
    quad_form = torch.matmul(data_vec.T, v)

    # 5. Calculate the NLL
    # NLL = 0.5 * (N*log(2*pi) + log(det(K)) + y^T * K^{-1} * y)
    log_2pi = torch.log(torch.tensor(2 * np.pi, device=device, dtype=K.dtype))
    nll = 0.5 * (N * log_2pi + log_det_K + quad_form)

    # Return the average NLL per point for stable optimization
    return nll / N


# =========================================================================
# 3. Training Loop (Adam)
# =========================================================================



def get_phi_params(log_params_list):
    """Helper to print reparameterized (phi-scale) params."""
    try:
        # p_list is a list of 1-element tensors
        p_cat = torch.cat([p.detach().clone().cpu() for p in log_params_list])
        # Exponentiate the log-params
        phi1 = torch.exp(p_cat[0])
        phi2 = torch.exp(p_cat[1])
        phi3 = torch.exp(p_cat[2])
        nugget = torch.exp(p_cat[3]) # This is already what you were doing
        
        return (f"phi1: {phi1.item():.4f}, phi2: {phi2.item():.4f}, "
                f"phi3: {phi3.item():.4f}, nugget: {nugget.item():.4f}")
    except Exception:
        return "[Error in reparam conversion]"
    
# =========================================================================
# 4. Main Simulation Script
# =========================================================================
if __name__ == '__main__':
    start_time = time.time()

    # --- 0. Configuration ---
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {DEVICE}")

    N1 = 25  # Number of latitude points
    N2 = 25  # Number of longitude points
    
    LAT_RANGE = (0.0, 5.0)
    LON_RANGE = (0.0, 5.0) 
    
    EPOCHS = 200
    LEARNING_RATE = 0.01 

    print(f"Grid: {N1}x{N2} ({N1*N2} points).")
    
    # --- 1. Define True Parameters ---
    TRUE_PARAMS = {
        'sigmasq': 30.0,
        'range_lat': 1.0,
        'range_lon': 1.5,
        'nugget': 0.1 
    }
    print(f"\n--- True Parameters (Natural Scale) ---")
    for key, val in TRUE_PARAMS.items():
        print(f"  {key:<10}: {val:.4f}")

    # --- Calculate and print true log-params ---
    true_log_params_tensor = convert_to_log_reparam(
        torch.tensor(TRUE_PARAMS['sigmasq']),
        torch.tensor(TRUE_PARAMS['range_lat']),
        torch.tensor(TRUE_PARAMS['range_lon']),
        torch.tensor(TRUE_PARAMS['nugget'])
    )
    # 💥 Store the true log params as a list of tensors for the helper
    true_log_params_list = [Parameter(p.unsqueeze(0)) for p in true_log_params_tensor]
    
    # --- 2. Generate Simulated Data ---
    data_vec, locs = generate_spatial_data(
        N1, N2, LAT_RANGE, LON_RANGE, TRUE_PARAMS, DEVICE
    )
    
    data_vec_dev = data_vec.to(DEVICE)
    locs_dev = locs.to(DEVICE)

    # --- 3. Optimization (Using Adam) ---
    INIT_PARAMS = {
        'sigmasq': 25.0,
        'range_lat': 0.8,
        'range_lon': 1.2,
        'nugget': 0.2 
    }
    print(f"\n--- Initial Parameters for Fitting ---")
    print(INIT_PARAMS)
    
    log_params_init = convert_to_log_reparam(
        torch.tensor(INIT_PARAMS['sigmasq']),
        torch.tensor(INIT_PARAMS['range_lat']),
        torch.tensor(INIT_PARAMS['range_lon']),
        torch.tensor(INIT_PARAMS['nugget'])
    ).float()

    params_list = [
        Parameter(torch.tensor([val], dtype=torch.float32, device=DEVICE))
        for val in log_params_init
    ]

    optimizer = torch.optim.Adam(params_list, lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
    
    best_loss = float('inf')
    best_params_state = None

    print("\n--- Starting Full MLE Optimization (No Tapering) ---")
    for epoch in range(EPOCHS):
        optimizer.zero_grad()
        params_tensor = torch.cat(params_list)
        
        loss = full_nll_loss(params_tensor, data_vec_dev, locs_dev)

        if torch.isnan(loss) or torch.isinf(loss):
            print(f"Epoch {epoch+1}/{EPOCHS}: Loss is NaN/Inf. Stopping optimization.")
            if best_params_state is None: 
                 best_params_state = [p.detach().clone() for p in params_list]
            break
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(params_list, max_norm=1.0)
        optimizer.step()
        scheduler.step()
        
        current_loss_item = loss.item()
        
        if current_loss_item < best_loss:
            params_valid = not any(torch.isnan(p.data).any() or torch.isinf(p.data).any() for p in params_list)
            if params_valid:
                best_loss = current_loss_item
                best_params_state = [p.detach().clone() for p in params_list]

        if epoch % 20 == 0 or epoch == EPOCHS - 1:
            print(f'Epoch {epoch+1}/{EPOCHS} | Loss: {current_loss_item:.4f} | LR: {scheduler.get_last_lr()[0]:.6f}')
            if epoch % 50 == 0 or epoch == EPOCHS - 1:
                 print(f'  Params: {get_printable_params(params_list)}')

    print("\n--- Optimization Complete ---")

    # --- 4. Report Results ---
    print(f"\n\n{'='*25} FINAL SIMULATION RESULTS {'='*25}")
    print(f"Grid Size: {N1} lats x {N2} lons ({N1*N2} points)")
    
    print("\n--- True Parameters (Natural Scale) ---")
    for key, val in TRUE_PARAMS.items():
        print(f"  {key:<10}: {val:.4f}")
        
    print("\n--- Estimated Parameters (Natural Scale) ---")
    if best_params_state:
        final_params_str = get_printable_params(best_params_state)
        print(f"  Final Loss: {best_loss:.4f}")
        print(f"  Params: {final_params_str}")
    else:
        print("Optimization failed to produce valid results.")

    # --- 💥 REVISED: Print reparameterized results 💥 ---
    print("\n--- True Reparameterized Parameters (Phi Scale) ---")
    # Use the new helper on the true log params list
    print(f"  Params: {get_phi_params(true_log_params_list)}") 
    
    print("\n--- Estimated Reparameterized Parameters (Phi Scale) ---")
    if best_params_state:
        # Use the new helper on the final estimated params
        final_reparam_str = get_phi_params(best_params_state) 
        print(f"  Params: {final_reparam_str}")
    else:
        print("Optimization failed to produce valid results.")
    # --- END REVISED ---

    end_time = time.time()
    print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")

Using device: cpu
Grid: 25x25 (625 points).

--- True Parameters (Natural Scale) ---
  sigmasq   : 30.0000
  range_lat : 1.0000
  range_lon : 1.5000
  nugget    : 0.1000
Generating spatial data (25 lats x 25 lons)...
Building N x N covariance matrix (N=625)...
Performing Cholesky decomposition...
Data generated.

--- Initial Parameters for Fitting ---
{'sigmasq': 25.0, 'range_lat': 0.8, 'range_lon': 1.2, 'nugget': 0.2}

--- Starting Full MLE Optimization (No Tapering) ---
Epoch 1/200 | Loss: 2.3033 | LR: 0.009999
  Params: sigmasq: 24.5050, range_lat: 0.7881, range_lon: 1.1881, nugget: 0.1980
Epoch 21/200 | Loss: 2.3029 | LR: 0.009730
Epoch 41/200 | Loss: 2.3028 | LR: 0.008999
Epoch 61/200 | Loss: 2.3028 | LR: 0.007875
Epoch 81/200 | Loss: 2.3028 | LR: 0.006471
Epoch 101/200 | Loss: 2.3028 | LR: 0.004922
  Params: sigmasq: 21.7264, range_lat: 0.7025, range_lon: 1.0993, nugget: 0.2689
Epoch 121/200 | Loss: 2.3028 | LR: 0.003381
Epoch 141/200 | Loss: 2.3028 | LR: 0.001999
Epoch 161/200 |

no tapering below

In [28]:
import torch
import numpy as np
import matplotlib.pyplot as plt 
import cmath
import pickle
import time 
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.nn import Parameter
import torch.fft 
from typing import List, Dict, Any, Callable

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# =========================================================================
# 1. Data Generation and Model Functions
# =========================================================================

def convert_to_log_reparam(sigmasq, range_lat, range_lon, nugget):
    """
    Converts physical parameters to the 4-parameter log-reparameterization.
    """
    device = sigmasq.device
    epsilon = 1e-12
    
    range_lon_inv = 1.0 / (range_lon + epsilon)
    log_phi2 = torch.log(range_lon_inv)
    log_phi1 = torch.log(sigmasq * range_lon_inv + epsilon)
    log_phi3 = torch.log((range_lon / range_lat).pow(2) + epsilon)
    log_nugget = torch.log(nugget + epsilon)
    
    log_params = torch.tensor([log_phi1, log_phi2, log_phi3, log_nugget], device=device)
    return log_params


def build_cov_matrix(locs, log_params):
    """
    Builds the full N x N covariance matrix from the SpatialModel's
    4-parameter reparameterization.
    
    locs: (N, 2) tensor of [lat, lon] coordinates
    log_params: (4,) tensor of [log_phi1, log_phi2, log_phi3, log_nugget]
    """
    device = locs.device
    
    # --- A. Recover all parameters ---
    phi1   = torch.exp(log_params[0])
    phi2   = torch.exp(log_params[1]) # This is range_lon_inv
    phi3   = torch.exp(log_params[2]) # This is (range_lon / range_lat)^2
    nugget = torch.exp(log_params[3])

    # --- B. Derive Physical Parameters ---
    epsilon = 1e-12
    phi2_safe = phi2 + epsilon 
    sigmasq = phi1 / phi2_safe
    range_lon_inv = phi2
    range_lat_inv = torch.sqrt(phi3 + epsilon) * phi2

    # --- C. Calculate Anisotropic Spatial Distance ---
    x_lat, x_lon = locs[:, 0], locs[:, 1]
    
    delta_lat = x_lat.unsqueeze(1) - x_lat.unsqueeze(0)
    delta_lon = x_lon.unsqueeze(1) - x_lon.unsqueeze(0)

    dist_sq = (delta_lat * range_lat_inv).pow(2) + (delta_lon * range_lon_inv).pow(2)
    distance = torch.sqrt(dist_sq + epsilon) 

    # --- D. Calculate Covariance (Matern 0.5 = Exponential) ---
    cov = sigmasq * torch.exp(-distance)

    # --- E. Add Nugget ---
    cov.diagonal().add_(nugget + epsilon) # Add jitter to nugget
    
    return cov


def generate_spatial_data(n1, n2, lat_range, lon_range, true_params, device):
    """
    Generates a single n1 x n2 spatial field.
    """
    print(f"Generating spatial data ({n1} lats x {n2} lons)...")
    N = n1 * n2
    
    lats = torch.linspace(lat_range[0], lat_range[1], n1, device=device, dtype=torch.float64)
    lons = torch.linspace(lon_range[0], lon_range[1], n2, device=device, dtype=torch.float64)
    grid_lats, grid_lons = torch.meshgrid(lats, lons, indexing='ij')
    
    locs = torch.stack([grid_lats.ravel(), grid_lons.ravel()], dim=1)
    
    log_params_true = convert_to_log_reparam(
        torch.tensor(true_params['sigmasq'], device=device, dtype=torch.float64), 
        torch.tensor(true_params['range_lat'], device=device, dtype=torch.float64),
        torch.tensor(true_params['range_lon'], device=device, dtype=torch.float64),
        torch.tensor(true_params['nugget'], device=device, dtype=torch.float64)
    )
    
    print("Building N x N covariance matrix (N={})... This may take a moment.".format(N))
    K = build_cov_matrix(locs, log_params_true)
    
    print("Performing Cholesky decomposition...")
    try:
        L = torch.linalg.cholesky(K)
    except torch.linalg.LinAlgError:
        print("Warning: Cholesky failed. Adding jitter.")
        L = torch.linalg.cholesky(K + torch.eye(N, device=device, dtype=torch.float64) * 1e-5)
        
    z = torch.randn(N, 1, device=device, dtype=torch.float64)
    data_vec = (L @ z).float() # Convert to float32 for FFT/Likelihood
    
    print("Data generated.")
    return data_vec.reshape(n1, n2)


# =========================================================================
# 2. Tapering, Autocorrelation, and Covariance Functions
# =========================================================================

def cgn_hamming(u, n1, n2):
    """Computes a 2D Hamming window."""
    u1, u2 = u
    device = u1.device if isinstance(u1, torch.Tensor) else (u2.device if isinstance(u2, torch.Tensor) else torch.device('cpu'))
    u1_tensor = u1.to(device) if isinstance(u1, torch.Tensor) else torch.tensor(u1, device=device, dtype=torch.float32)
    u2_tensor = u2.to(device) if isinstance(u2, torch.Tensor) else torch.tensor(u2, device=device, dtype=torch.float32)

    n1_eff = float(n1) if n1 > 0 else 1.0
    n2_eff = float(n2) if n2 > 0 else 1.0
    hamming1 = 0.54 + 0.46 * torch.cos(2.0 * torch.pi * u1_tensor / n1_eff)
    hamming2 = 0.54 + 0.46 * torch.cos(2.0 * torch.pi * u2_tensor / n2_eff)
    return hamming1 * hamming2

# --- 💥 NEW TAPERING FUNCTION 💥 ---
def cgn_rectangular(u, n1, n2):
    """Computes a 2D Rectangular window (i.e., no taper)."""
    # u is a tuple of meshgrids (u1_mesh, u2_mesh)
    u1_mesh, _ = u
    return torch.ones_like(u1_mesh)
# --- END NEW FUNCTION ---

def calculate_taper_autocorrelation_fft(taper_grid, n1, n2, device):
    """
    Computes the normalized taper autocorrelation function c_gn(u) using FFT.
    This function is generic and works for any taper_grid, including rectangular.
    """
    taper_grid = taper_grid.to(device) 
    H = torch.sum(taper_grid**2)
    if H < 1e-12:
        print("Warning: Sum of squared taper weights (H) is near zero.")
        return torch.zeros((2*n1-1, 2*n2-1), device=device, dtype=taper_grid.dtype)

    N1, N2 = 2 * n1 - 1, 2 * n2 - 1
    taper_fft = torch.fft.fft2(taper_grid, s=(N1, N2))
    power_spectrum = torch.abs(taper_fft)**2
    autocorr_unnormalized = torch.fft.ifft2(power_spectrum).real
    autocorr_shifted = torch.fft.fftshift(autocorr_unnormalized)
    c_gn_grid = autocorr_shifted / (H + 1e-12)
    return c_gn_grid 

# =========================================================================
# 3. Covariance & Spectral Density
# =========================================================================

# --- DEFINE DELTAS GLOBALLY (or within function) ---
DELTA_LAT = 0.044
DELTA_LON = 0.063


def cov_x_spatial_model_kernel(u1, u2, t, params):
    """
    Computes autocovariance of X using the SPATIAL-ONLY kernel.
    u1, u2 are PHYSICAL lags (already scaled by deltas).
    """
    device = params.device
    u1_dev = u1.to(device) if isinstance(u1, torch.Tensor) else torch.tensor(u1, device=device, dtype=torch.float32)
    u2_dev = u2.to(device) if isinstance(u2, torch.Tensor) else torch.tensor(u2, device=device, dtype=torch.float32)
    t_dev = t.to(device) if isinstance(t, torch.Tensor) else torch.tensor(t, device=device, dtype=torch.float32)

    if torch.isnan(params).any() or torch.isinf(params).any():
         out_shape = torch.broadcast_shapes(u1_dev.shape, u2_dev.shape, t_dev.shape)
         return torch.full(out_shape, float('nan'), device=device, dtype=torch.float32)

    phi1   = torch.exp(params[0])
    phi2   = torch.exp(params[1]) 
    phi3   = torch.exp(params[2])
    nugget = torch.exp(params[3])

    epsilon = 1e-12
    sigmasq = phi1 / (phi2 + epsilon)  
    range_lon_inv = phi2
    range_lat_inv = torch.sqrt(phi3 + epsilon) * phi2

    dist_sq = (u1_dev * range_lat_inv).pow(2) + (u2_dev * range_lon_inv).pow(2)
    distance = torch.sqrt(dist_sq + epsilon) 

    cov_smooth = sigmasq * torch.exp(-distance)

    is_zero_lag = (torch.abs(u1_dev) < 1e-9) & (torch.abs(u2_dev) < 1e-9) & (torch.abs(t_dev) < 1e-9)
    final_cov = torch.where(is_zero_lag, cov_smooth + nugget, cov_smooth)

    return final_cov


def cn_bar_tapered(u1, u2, t, params, n1, n2, taper_autocorr_grid):
    """
    Computes c_X(u) * c_gn(u).
    u1, u2 are GRID index lags (e.g., -n1..0..n1)
    """
    device = params.device
    u1_dev = u1.to(device) if isinstance(u1, torch.Tensor) else torch.tensor(u1, device=device, dtype=torch.float32)
    u2_dev = u2.to(device) if isinstance(u2, torch.Tensor) else torch.tensor(u2, device=device, dtype=torch.float32)
    t_dev = t.to(device) if isinstance(t, torch.Tensor) else torch.tensor(t, device=device, dtype=torch.float32)

    # --- KEY CORRECTION: Scale lags by deltas ---
    lag_u1 = u1_dev * DELTA_LAT
    lag_u2 = u2_dev * DELTA_LON

    # Call kernel with true spatial lags
    cov_X_value = cov_x_spatial_model_kernel(lag_u1, lag_u2, t_dev, params)

    # --- Get Taper Autocorrelation Value c_gn(u) from grid ---
    u1_idx = u1_dev.long()
    u2_idx = u2_dev.long()
    idx1 = (n1 - 1 + u1_idx)
    idx2 = (n2 - 1 + u2_idx)
    idx1 = torch.clamp(idx1, 0, 2 * n1 - 2)
    idx2 = torch.clamp(idx2, 0, 2 * n2 - 2)

    taper_autocorr_value = taper_autocorr_grid[idx1, idx2] 

    if torch.isnan(cov_X_value).any() or torch.isnan(taper_autocorr_value).any():
        out_shape = torch.broadcast_shapes(cov_X_value.shape, taper_autocorr_value.shape)
        return torch.full(out_shape, float('nan'), device=device, dtype=torch.float32)

    result = cov_X_value * taper_autocorr_value
    return result


def expected_periodogram_fft_tapered(params, n1, n2, p, taper_autocorr_grid):
    """
    Calculates the expected periodogram using the aliasing sum (Lemma 2).
    """
    device = params.device if isinstance(params, torch.Tensor) else params[0].device
    if isinstance(params, list):
        params_tensor = torch.cat([p.to(device) for p in params])
    else:
        params_tensor = params.to(device)

    # 1. Create the grid for u from [0, n-1]
    u1_lags = torch.arange(n1, dtype=torch.float32, device=device)
    u2_lags = torch.arange(n2, dtype=torch.float32, device=device)
    u1_mesh, u2_mesh = torch.meshgrid(u1_lags, u2_lags, indexing='ij')

    t_lags = torch.arange(p, dtype=torch.float32, device=device)
    tilde_cn_tensor = torch.zeros((n1, n2, p, p), dtype=torch.complex64, device=device)

    # 2. Loop over the p x p matrix (for this spatial code, p=1)
    for q in range(p):
        for r in range(p):
            t_diff = t_lags[q] - t_lags[r]
            
            # --- Calculate 4 aliasing terms from Lemma 2 ---
            term1 = cn_bar_tapered(u1_mesh, u2_mesh, t_diff, 
                                   params_tensor, n1, n2, taper_autocorr_grid)
            term2 = cn_bar_tapered(u1_mesh - n1, u2_mesh, t_diff, 
                                   params_tensor, n1, n2, taper_autocorr_grid)
            term3 = cn_bar_tapered(u1_mesh, u2_mesh - n2, t_diff, 
                                   params_tensor, n1, n2, taper_autocorr_grid)
            term4 = cn_bar_tapered(u1_mesh - n1, u2_mesh - n2, t_diff,
                                   params_tensor, n1, n2, taper_autocorr_grid)
            
            # 3. Sum them to create tilde_c_n(u, t_diff)
            tilde_cn_grid_qr = (term1 + term2 + term3 + term4)
            
            if torch.isnan(tilde_cn_grid_qr).any():
                 tilde_cn_tensor[:, :, q, r] = float('nan')
            else:
                 tilde_cn_tensor[:, :, q, r] = tilde_cn_grid_qr.to(torch.complex64)

    if torch.isnan(tilde_cn_tensor).any():
        print("Warning: NaN in product_tensor before FFT.")
        nan_shape = (n1, n2, p, p)
        return torch.full(nan_shape, float('nan'), dtype=torch.complex64, device=device)

    # 4. Compute FFT of the *aliased* covariance
    fft_result = torch.fft.fft2(tilde_cn_tensor, dim=(0, 1))
    fft_result_real = fft_result.real 
    normalization_factor = 1.0 / (4.0 * cmath.pi**2)
    result = fft_result_real * normalization_factor

    return result

# =========================================================================
# 4. Data Processing (Whittle)
# =========================================================================
def generate_Jvector_tapered(data_grid, taper_grid, device):
    """
    Generates J-vector for a single component using the specified taper.
    """
    p = 1 # This is a spatial-only model
    n1, n2 = data_grid.shape
    
    data_grid_tapered = data_grid.to(device) * taper_grid.to(device)

    if torch.isnan(data_grid_tapered).any() or torch.isinf(data_grid_tapered).any():
         print("Warning: NaN/Inf detected in data_grid_tapered before FFT. Replacing with zeros.")
         data_grid_tapered = torch.nan_to_num(data_grid_tapered, nan=0.0, posinf=0.0, neginf=0.0)

    J_vec_spatial = torch.fft.fft2(data_grid_tapered)
    
    # Add 'p' dimension
    J_vector_tensor = J_vec_spatial.unsqueeze(-1)

    H = torch.sum(taper_grid**2)
    if H < 1e-12:
        print("Warning: Normalization factor H is near zero.")
        norm_factor = torch.tensor(0.0, device=device)
    else:
        norm_factor = (torch.sqrt(1.0 / H) / (2.0 * cmath.pi)).to(device)

    result = J_vector_tensor * norm_factor
    if torch.isnan(result).any(): print("Warning: NaN in J_vector output.")
    return result, n1, n2, p


def calculate_sample_periodogram_vectorized(J_vector_tensor):
    """Calculates sample periodogram I_n = J J^H (pxp matrix for each spatial freq)."""
    if torch.isnan(J_vector_tensor).any() or torch.isinf(J_vector_tensor).any():
        print("Warning: NaN/Inf detected in J_vector_tensor input.")
        n1, n2, p = J_vector_tensor.shape
        return torch.full((n1, n2, p, p), float('nan'), dtype=torch.complex64, device=J_vector_tensor.device)

    J_col = J_vector_tensor.unsqueeze(-1)
    J_row_conj = J_vector_tensor.unsqueeze(-2).conj()
    result = J_col @ J_row_conj

    if torch.isnan(result).any(): print("Warning: NaN in periodogram matrix output.")
    return result


# =========================================================================
# 5. Likelihood Calculation (Whittle)
# =========================================================================

def whittle_likelihood_loss_tapered(params, I_sample, n1, n2, p, taper_autocorr_grid):
    """
    Whittle Likelihood Loss using data tapering and exact taper autocorrelation c_gn.
    """
    device = I_sample.device
    params_tensor = params.to(device)

    if torch.isnan(params_tensor).any() or torch.isinf(params_tensor).any():
        print("Warning: NaN/Inf detected in input parameters to likelihood.")
        return torch.tensor(float('nan'), device=device)

    I_expected = expected_periodogram_fft_tapered(
        params_tensor, n1, n2, p, taper_autocorr_grid
    )

    if torch.isnan(I_expected).any() or torch.isinf(I_expected).any():
        print("Warning: NaN/Inf returned from expected_periodogram calculation.")
        return torch.tensor(float('nan'), device=device)

    eye_matrix = torch.eye(p, dtype=torch.complex64, device=device)
    diag_vals = torch.abs(I_expected.diagonal(dim1=-2, dim2=-1))
    mean_diag_abs = diag_vals.mean().item() if diag_vals.numel() > 0 and not torch.isnan(diag_vals).all() else 1.0
    diag_load = max(mean_diag_abs * 1e-8, 1e-9)
    I_expected_stable = I_expected + eye_matrix * diag_load

    sign, logabsdet = torch.linalg.slogdet(I_expected_stable)
    if torch.any(sign.real <= 1e-9):
        print("Warning: Non-positive determinant encountered. Applying penalty.")
        log_det_term = torch.where(sign.real > 1e-9, logabsdet, torch.tensor(1e10, device=device))
    else:
        log_det_term = logabsdet

    if torch.isnan(I_sample).any() or torch.isinf(I_sample).any():
        print("Warning: NaN/Inf detected in I_sample input to likelihood.")
        return torch.tensor(float('nan'), device=device)

    try:
        solved_term = torch.linalg.solve(I_expected_stable, I_sample)
        trace_term = torch.einsum('...ii->...', solved_term).real
    except torch.linalg.LinAlgError as e:
        print(f"Warning: LinAlgError during solve: {e}. Applying high loss penalty.")
        return torch.tensor(float('inf'), device=device)

    if torch.isnan(trace_term).any() or torch.isinf(trace_term).any():
        print("Warning: NaN/Inf detected in trace_term. Returning NaN loss.")
        return torch.tensor(float('nan'), device=device)

    likelihood_terms = log_det_term + trace_term

    if torch.isnan(likelihood_terms).any():
        print("Warning: NaN detected in likelihood_terms before summation. Returning NaN loss.")
        return torch.tensor(float('nan'), device=device)

    total_sum = torch.sum(likelihood_terms)
    # Exclude the DC component (frequency 0,0)
    dc_term = likelihood_terms[0, 0] if n1 > 0 and n2 > 0 else torch.tensor(0.0, device=device)
    if torch.isnan(dc_term).any() or torch.isinf(dc_term).any():
        print("Warning: NaN/Inf detected in DC term. Setting to 0.")
        dc_term = torch.tensor(0.0, device=device)

    loss = total_sum - dc_term if (n1 > 1 or n2 > 1) else total_sum

    if torch.isnan(loss) or torch.isinf(loss):
         print("Warning: NaN/Inf detected in final loss. Returning Inf penalty.")
         return torch.tensor(float('inf'), device=device)

    return loss


# =========================================================================
# 6. Training Loop Helpers
# =========================================================================

def get_printable_params(p_list):
    """Helper to convert log-params to natural scale for printing."""
    p_cat = torch.cat([p.detach().clone().cpu() for p in p_list])
    if torch.isnan(p_cat).any() or torch.isinf(p_cat).any():
        return "[NaN/Inf in log_params]"
    try:
        phi1, phi2, phi3, nugget_log = [torch.exp(p) for p in p_cat]
        epsilon = 1e-12
        sigmasq = phi1 / (phi2 + epsilon)
        range_lon = 1.0 / (phi2 + epsilon)
        range_lat = 1.0 / (torch.sqrt(phi3 + epsilon) * phi2 + epsilon)
        nugget = nugget_log # This is exp(log_nugget)
        
        return (f"sigmasq: {sigmasq.item():.4f}, range_lat: {range_lat.item():.4f}, "
                f"range_lon: {range_lon.item():.4f}, nugget: {nugget.item():.4f}")
    except Exception:
        return "[Error in param conversion]"

# --- 💥 NEW HELPER 💥 ---
def get_reparam_params(log_params_list):
    """Helper to print reparameterized (log-scale) params."""
    try:
        p_cat = torch.cat([p.detach().clone().cpu() for p in log_params_list])
        return (f"log_phi1: {p_cat[0].item():.4f}, log_phi2: {p_cat[1].item():.4f}, "
                f"log_phi3: {p_cat[2].item():.4f}, log_nugget: {p_cat[3].item():.4f}")
    except Exception:
        return "[Error in reparam conversion]"
# --- END NEW HELPER ---


# =========================================================================
# 7. Main Simulation Script
# =========================================================================
if __name__ == '__main__':
    start_time = time.time()

    # --- 0. Configuration ---
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {DEVICE}")

    N1 = 115  # Number of latitude points
    N2 = 160  # Number of longitude points
    
    LAT_RANGE = (0.0, 5.016)
    LON_RANGE = (123.0, 132.997) 
    
    P = 1 # Spatial-only
    EPOCHS = 1000
    LEARNING_RATE = 0.005 

    print(f"Grid: {N1}x{N2} ({N1*N2} points).")
    print(f"Lat Range: {LAT_RANGE}. Lon Range: {LON_RANGE}.")
    print(f"Deltas (hard-coded in likelihood): {DELTA_LAT}, {DELTA_LON}")

    
    # --- 1. Define True Parameters ---
    TRUE_PARAMS = {
        'sigmasq': 30.0,
        'range_lat': 1.0,
        'range_lon': 1.5,
        'nugget': 0.1 
    }
    print(f"\n--- True Parameters (Natural Scale) ---")
    for key, val in TRUE_PARAMS.items():
        print(f"  {key:<10}: {val:.4f}")
        
    # --- 💥 NEW: Calculate and print true log-params 💥 ---
    true_log_params_tensor = convert_to_log_reparam(
        torch.tensor(TRUE_PARAMS['sigmasq']),
        torch.tensor(TRUE_PARAMS['range_lat']),
        torch.tensor(TRUE_PARAMS['range_lon']),
        torch.tensor(TRUE_PARAMS['nugget'])
    )
    print("\n--- True Reparameterized Parameters ---")
    true_log_params_list = [Parameter(p.unsqueeze(0)) for p in true_log_params_tensor]
    print(f"  {get_reparam_params(true_log_params_list)}")
    # --- END NEW ---

    # --- 2. Generate Simulated Data ---
    print("WARNING: Building an 18400 x 18400 covariance matrix.")
    data_grid = generate_spatial_data(
        N1, N2, LAT_RANGE, LON_RANGE, TRUE_PARAMS, DEVICE
    )
    
    # --- 3. Pre-compute for Whittle Likelihood ---
    print("\nPre-computing Taper, Autocorrelation, and Periodogram...")
    
    # --- 💥 MODIFICATION: Use rectangular taper 💥 ---
    TAPERING_FUNC = cgn_rectangular
    print("Using RECTANGULAR taper (i.e., no tapering).")
    # --- END MODIFICATION ---
    
    u1_mesh_cpu, u2_mesh_cpu = torch.meshgrid(
        torch.arange(N1, dtype=torch.float32),
        torch.arange(N2, dtype=torch.float32),
        indexing='ij'
    )
    taper_grid = TAPERING_FUNC((u1_mesh_cpu, u2_mesh_cpu), N1, N2).to(DEVICE)
    taper_autocorr_grid = calculate_taper_autocorrelation_fft(taper_grid, N1, N2, DEVICE)

    # Compute J-vector (FFT of tapered data)
    J_vec, n1_check, n2_check, p_check = generate_Jvector_tapered(data_grid, taper_grid, DEVICE)
    
    I_sample = calculate_sample_periodogram_vectorized(J_vec) # (N1, N2, 1, 1)
    
    if torch.isnan(I_sample).any() or torch.isinf(I_sample).any():
        print("Error: NaN/Inf in sample periodogram. Exiting.")
        exit()
    if torch.isnan(taper_autocorr_grid).any() or torch.isinf(taper_autocorr_grid).any():
        print("Error: NaN/Inf in taper autocorrelation. Exiting.")
        exit()

    print(f"Pre-computation complete.")

    # --- 4. Optimization (Using Adam) ---
    INIT_PARAMS = {
        'sigmasq': 25.0,
        'range_lat': 0.8,
        'range_lon': 1.2,
        'nugget': 0.2 
    }
    print(f"\n--- Initial Parameters for Fitting ---")
    print(INIT_PARAMS)
    
    log_params_init = convert_to_log_reparam(
        torch.tensor(INIT_PARAMS['sigmasq']),
        torch.tensor(INIT_PARAMS['range_lat']),
        torch.tensor(INIT_PARAMS['range_lon']),
        torch.tensor(INIT_PARAMS['nugget'])
    ).float()

    params_list = [
        Parameter(torch.tensor([val], dtype=torch.float32, device=DEVICE))
        for val in log_params_init
    ]

    optimizer = torch.optim.Adam(params_list, lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
    
    best_loss = float('inf')
    best_params_state = None

    I_sample_dev = I_sample.to(DEVICE)
    taper_autocorr_grid_dev = taper_autocorr_grid.to(DEVICE)

    print("\n--- Starting Adam Optimization (No Tapering) ---")
    for epoch in range(EPOCHS):
        optimizer.zero_grad()
        params_tensor = torch.cat(params_list)
        
        loss = whittle_likelihood_loss_tapered(
            params_tensor, I_sample_dev, N1, N2, P, 
            taper_autocorr_grid_dev
        )

        if torch.isnan(loss) or torch.isinf(loss):
            print(f"Epoch {epoch+1}/{EPOCHS}: Loss is NaN/Inf. Stopping optimization.")
            if best_params_state is None:
                best_params_state = [p.detach().clone() for p in params_list]
            break
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(params_list, max_norm=1.0)
        optimizer.step()
        scheduler.step()
        
        current_loss_item = loss.item()
        
        if current_loss_item < best_loss:
            params_valid = not any(torch.isnan(p.data).any() or torch.isinf(p.data).any() for p in params_list)
            if params_valid:
                best_loss = current_loss_item
                best_params_state = [p.detach().clone() for p in params_list]

        if epoch % 50 == 0 or epoch == EPOCHS - 1:
            print(f'Epoch {epoch+1}/{EPOCHS} | Loss: {current_loss_item:.4f} | LR: {scheduler.get_last_lr()[0]:.6f}')
            if epoch % 200 == 0 or epoch == EPOCHS - 1:
                 print(f'  Params: {get_printable_params(params_list)}')


    print("\n--- Adam Optimization Complete ---")

    # --- 5. Report Results ---
    print(f"\n\n{'='*25} FINAL SIMULATION RESULTS {'='*25}")
    print(f"Grid Size: {N1} lats x {N2} lons ({N1*N2} points)")
    print("Method: Whittle Likelihood (No Tapering)")
    
    print("\n--- True Parameters (Natural Scale) ---")
    for key, val in TRUE_PARAMS.items():
        print(f"  {key:<10}: {val:.4f}")
        
    print("\n--- True Reparameterized Parameters ---")
    print(f"  {get_reparam_params(true_log_params_list)}")
        
    print("\n--- Estimated Parameters (Natural Scale) ---")
    if best_params_state:
        final_params_str = get_printable_params(best_params_state)
        print(f"  Final Loss: {best_loss:.4f}")
        print(f"  Final Params: {final_params_str}")
    else:
        print("Optimization failed to produce valid results.")

    # --- 💥 NEW: Print estimated reparam params 💥 ---
    print("\n--- Estimated Reparameterized Parameters ---")
    if best_params_state:
        final_reparam_str = get_reparam_params(best_params_state)
        print(f"  Final Reparam Params: {final_reparam_str}")
    else:
        print("Optimization failed to produce valid results.")
    # --- END NEW ---
    
    end_time = time.time()
    print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")

Using device: cpu
Grid: 115x160 (18400 points).
Lat Range: (0.0, 5.016). Lon Range: (123.0, 132.997).
Deltas (hard-coded in likelihood): 0.044, 0.063

--- True Parameters (Natural Scale) ---
  sigmasq   : 30.0000
  range_lat : 1.0000
  range_lon : 1.5000
  nugget    : 0.1000

--- True Reparameterized Parameters ---
  log_phi1: 2.9957, log_phi2: -0.4055, log_phi3: 0.8109, log_nugget: -2.3026
Generating spatial data (115 lats x 160 lons)...
Building N x N covariance matrix (N=18400)... This may take a moment.
Performing Cholesky decomposition...
Data generated.

Pre-computing Taper, Autocorrelation, and Periodogram...
Using RECTANGULAR taper (i.e., no tapering).
Pre-computation complete.

--- Initial Parameters for Fitting ---
{'sigmasq': 25.0, 'range_lat': 0.8, 'range_lon': 1.2, 'nugget': 0.2}

--- Starting Adam Optimization (No Tapering) ---
Epoch 1/1000 | Loss: -39207.4883 | LR: 0.005000
  Params: sigmasq: 24.7512, range_lat: 0.7980, range_lon: 1.1940, nugget: 0.1990
Epoch 51/1000 | L